In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot

fm3d

In [2]:
earth_radius=6371

interface_tolerance=0.1

n_interfaces=11
n_intersections=n_interfaces

n_vgrids=10
n_vtypes=10

n_sources=10
n_receivers=100

n_sources_ppinc=n_sources
# is_teleseismic=0
n_teleseismic=0

class rays():
    def __init__(self):
        self.raypath_id=0

class source(rays):
    def __init__(self):
        self.is_teleseismic=False
        self.is_local=True
        self.teleseismic_phase=0
        self.r=0
        self.long=0
        self.lat=0
        self.coslat=0
        self.ir=0
        self.ilat=0
        self.ilong=0
#         self.
        
class path(source):
    def __init(self):
        self.id=0
        self.n_tf=0
        self.sequence=np.zeros(2*self.n_tf)
        self.tf_sequence=np.zeros(2*self.n_tf)
        self.vtype_sequence=np.zeros(2*self.n_tf)
        self.fitting_interface=0
        self.refstep=0

class receivers():
    def __init__(self):
        self.n_rays=0
        self.source_equivalent=0
        
class velgrid():
    def __init__(self):
        self.nr=0
        self.nlat=0
        self.nlong=0
        self.r0=0
        self.lat0=0
        self.long0=0
        self.dr0=0
        self.dlat0=0
        self.dlong0=0
        self.r=np.zeros(self.nr)
        self.lat=np.zeros(self.nlat)
        self.long=np.zeros(self.nlong)
        self.nnode=0
        #         self.velocity=10

class propgrid():
    def __init__(self):
        self.nr=10
        self.nlat=10
        self.nlong=10
        self.r0=0
        self.lat0=0
        self.long0=0
        self.dr0=0
        self.dlat0=0
        self.dlong0=0
        self.r=np.zeros(self.nr)
        self.lat=np.zeros(self.nlat)
        self.long=np.zeros(self.nlong)
        self.tolerance=0

class region():
    def __init__(self):
        self.id_=0
        self.i_top=0
        self.i_bot=0
        
class intersections():
    def __init__(self):
        self.id_=0
        
intersection=[intersections() for i in range(n_intersections)]
        
class interfaces():
    def __init__(self):
        self.id=0
        self.nlat=10
        self.nlong=10
        self.lat0=0
        self.long0=0
        self.dlat=0
        self.dlong=0
        self.lat=np.zeros(self.nlat)
        self.long=np.zeros(self.nlong)
        self.r=np.ones((self.nlat,self.nlong))
        self.pinched=False ##(Doubt)

interface=[interfaces() for i in range(n_interfaces)]

#class intrface():
#     def __init__(self):

vgrid=[[velgrid() for i in range(n_vgrids)] for j in range(n_vtypes)]
pgrid=propgrid()

In [4]:
def sweep_region_from_interface(reg,istart_in,vtype,s):
    istart = istart_in
    itop = reg.itop
    ibot = reg.ibot
    
    received_turning_ray=False
    
    reg.node_status=-1
    
    for i in range(0,istart.nnode):
        istart.starttime[i]=istart.arrivaltime[i]
        if (istart.id == ibot.d):
            j=istart.rabo_node_id[i]
        else:
            j=istart.rbel_node_id[i]
    
    reg.arrivaltime[j]= istart.arrivaltime[i]
    reg.time_gradient[:3][j]=istart.time_gradient[:3][j]
    reg.node_status[j]=1
    
    propagate(reg,vtype)
    
    n_turning=0
    
    for i in range(0,reg.nnode):
        i1 = reg.node[i].i1
        i2 = reg.node[i].i2
        i3 = reg.node[i].i3
        if (i1 == 0):
            if (i2 == itop.id):
                itop.arrivaltime[i3] = reg.arrivaltime[i]
                itop.time_gradient[:3][i3] = reg.time_gradient[:3][i]
            else:
                ibot.arrivaltime[i3] = reg.arrivaltime[i]
                ibot.time_gradient[:3][i3] = reg.time_gradient[:3][i]
                
            if (i2 == istart.id):
                if (istart.arrivaltime[i3] < istart.starttime[i3]):
                    n_turning+=1
                    received_turning_ray[i3] = True
    
    s.n_time_fields+=1
    s.time_field[s.n_time_fields].arrivaltime=reg.arrivaltime
    
    tg=reg.time_gradient
    s.time_field[s.n_time_fields].time_gradient=tg
    
    if (n_turning > 0):
        s.time_field[s.n_time_fields].turning_rays_present = True
        s.time_field[s.n_time_fields].received_turning_ray = received_turning_ray
        
def sweep_sregion_from_interface(reg,istart_in,vtype):
    istart = istart_in
    itop = reg.itop
    ibot = reg.ibot
    
    reg.node_status=-1
    
    for i in range(0,istart.nnode):
        istart.starttime[i]=istart.arrivaltime[i]
        if (istart.id == ibot.id):
            j=istart.rabo_node_id[i]
        else:
            j=istart.rbel_node_id[i]
            
    reg.arrivaltime[j]= istart.arrivaltime[i]
    reg.time_gradient[:3][j]=istart.time_gradient[:3][i]
    reg.node_status[j]=1
    
    propagate(reg,vtype)
    
def sweep_region_from_source(reg,s,vtype):
    grid=reg.grid
    
    itop=reg.itop
    ibot=reg.ibot
    
    vel_source=interpolate_velocity(s.r,s.lat,s.long,vgrid(reg.ivgrid,vtype))

In [43]:
#         self.velocity=10
def initialise_velocity_grids(velgrid):
    for i in range(n_vtypes):
        for j in range(n_vgrids):
            for k in range(velgrid[i][j].nr):
                velgrid[i][j].r[k]=velgrid[i][j].r0 + k*velgrid[i][j].dr0
            for k in range(velgrid[i][j].nlat):
                velgrid[i][j].lat[k]=velgrid[i][j].lat0 + k*velgrid[i][j].dlat0
            for k in range(velgrid[i][j].nlong):
                velgrid[i][j].long[k]=velgrid[i][j].long0 + k*velgrid[i][j].long0
            for k in range(velgrid[i][j].nr):
                for l in range(velgrid[i][j].nlat):
                    for m in range(velgrid[i][j].nlong):
                        velgrid[i][j].velocity[k][l][m]
            velgrid[i][j].nnode=velgrid[i][j].nr*velgrid[i][j].nlat*velgrid[i][j].nlong

def initialise_propagation_grid(pgrid):
    df=pd.read_csv("pgrid.csv")
    pgrid.dr0=df['dr0'].to_numpy()[0]
    pgrid.dlat0=df['dlat0'].to_numpy()[0]
    pgrid.dlong0=df['dlong0'].to_numpy()[0]
    pgrid.lat0=df['lat0'].to_numpy()[0]
    pgrid.long0=df['long0'].to_numpy()[0]
    
    pgrid.nr=df['nr'].to_numpy()[0]
    pgrid.nlat=df['nlat'].to_numpy()[0]
    pgrid.nlong=df['nlong'].to_numpy()[0]
    
    pgrid.r0=earth_radius+(df['r0'].to_numpy()[0])-(pgrid.nr-1)*pgrid.dr0
#     pgrid.dlat0=np.radians(pgrid.dlat0)
#     pgrid.dlong0=np.radians(pgrid.dlong0)
#     pgrid.lat0=np.radians(pgrid.lat0)
#     pgrid.long0=np.radians(pgrid.long0)
    
    pgrid.tolerance=interface_tolerance*pgrid.dr0
    
    pgrid.rmax=pgrid.r0 + (pgrid.nr-1)*pgrid.dr0
    pgrid.latmax=pgrid.lat0 + (pgrid.nlat-1)*pgrid.dlat0
    pgrid.longmax=pgrid.long0 + (pgrid.nlong-1)*pgrid.dlong0
    
    for k in range(pgrid.nr):
        pgrid.r[k]=pgrid.r0 + k*pgrid.dr0
    for k in range(pgrid.nlat):
        pgrid.lat[k]=pgrid.lat0 + k*pgrid.dlat0
    for k in range(pgrid.nlong):
        pgrid.long[k]=pgrid.long0 + k*pgrid.dlong0
    pgrid.is_main_grid= True

def initialise_interfaces(intrface,n_interfaces):
    df=pd.read_csv("interfaces.csv")
    for i in range(n_interfaces):
#         interface_defaults(intrface[i])
        intrface[i].id=i
        intrface[i].nlat=df['nlat'].to_numpy()[i]   #read value
        intrface[i].nlong=df['nlong'].to_numpy()[i]
        intrface[i].dlat0=df['dlat0'].to_numpy()[i]
        intrface[i].dlong0=df['dlong0'].to_numpy()[i]
        intrface[i].lat0=df['lat0'].to_numpy()[i]
        intrface[i].long0=df['long0'].to_numpy()[i]
        intrface[i].r=df['r'].to_numpy()[i]*np.ones((intrface[i].nlat,intrface[i].nlong))
        intrface[i].lat=np.arange(intrface[i].lat0,(intrface[i].lat0+(intrface[i].nlat-1)*intrface[i].dlat0)+1,intrface[i].dlat0)
        intrface[i].long=np.arange(intrface[i].long0,(intrface[i].long0+(intrface[i].nlong-1)*intrface[i].dlong0)+1,intrface[i].dlong0)
        #         for j in range(intrface[i].nlat):
        #             intrface[i].lat[j]=intrface[i].lat0+i*intrface[i].dlat0
        #         for j in range(intrface[i].nlong):
        #             intrface[i].long[j]=intrface[i].long0+i*intrface[i].dlong0
        #         for j in range(intrface[i].nlat):
        #             for k in range(intrface[i].nlong):
        #                 intrface[i].r[j][k]= df['r'].to_numpy()[i] #read value
        intrface[i].nnode=intrface[i].nlat*intrface[i].nlong
        #error for surface under/over propagation grid Line- 241
    for i in range(n_interfaces): #(1,n_interfaces)
        for j in range(intrface[i].nlong):
            for k in range(intrface[i].nlat):
                hb=intrface[n_interfaces-1].r[j][k]
                if (intrface[i].r[j][k]<hb):
                    intrface[i].r[j][k]=hb
                    intrface[i].pinched=True
                    intrface[n_interfaces-1].pinched=True
                h=intrface[i].r[j][k]
                if (intrface[i].r[j][k]>h):
                    intrface[i].r[j][k]=h
                    intrface[i].pinched=True
                    intrface[n_interfaces-1].pinched=True

def interpolate_interface(lat,long,iface):
    ilat=int(np.floor((lat-iface.lat0)/iface.dlat0)+1)
    ilong=int(np.floor((long-iface.long0)/iface.dlong0)+1) #define iface

    if (ilong<2) or (ilong>(iface.nlong-2)):
        if (ilong<2):
            ilong=2
        elif (ilong>(iface.nlong-2)):
            ilong=(iface.nlong-2)
    if (ilat<2) or (ilat>(iface.nlat-2)):
        if (ilat<2):
            ilat=2
        elif (ilat>(iface.nlat-2)):
            ilat=(iface.nlat-2)

    u=(lat-iface.lat[ilat])/(iface.dlat0)
    v=(long-iface.long[ilong])/(iface.dlong0)
    bu=np.ones(4)
    bv=np.ones(4)

    bu[0]=((1-u)**3)/6
    bu[1]=(4-6*u**2+3*u**3)/6
    bu[2]=(1+3*u+3*u**2-3*u**3)/6
    bu[3]=u**3/6

    bv[0]=((1-v)**3)/6
    bv[1]=(4-6*v**2+3*v**3)/6
    bv[2]=(1+3*v+3*v**2-3*v**3)/6
    bv[3]=v**3/6

    value=0
    for i in range(4):
        for j in range(4):
            value+=bu[i]*bv[j]*iface.r[ilat+i-2][ilong+j-2]

    return value

def interface_normal():
    ilat=np.floor((lat-iface.lat0)/iface.dlat0)+1
    ilong=np.floor((long-iface.long0)/iface.dlong0)+1

    u=(lat-iface.lat[ilat])/(iface.dlat0)
    v=(long-iface.long[ilong])/(iface.dlong0)
    bu=np.ones(4)
    bv=np.ones(4)
    bpu=np.ones(4)
    bpv=np.ones(4)

    bu[0]=((1-u)**3)/6
    bu[1]=(4-6*u**2+3*u**3)/6
    bu[2]=(1+3*u+3*u**2-3*u**3)/6
    bu[3]=u**3/6

    bv[0]=((1-v)**3)/6
    bv[1]=(4-6*v**2+3*v**3)/6
    bv[2]=(1+3*v+3*v**2-3*v**3)/6
    bv[3]=v**3/6

    bpu[0]=-((1-u)**2)/2
    bpu[1]=-2*u+1.5*u**2
    bpu[2]=0.5+u-1.5*u**2
    bpu[3]=0.5*u**2

    bpv[0]=-((1-v)**2)/2
    bpv[1]=-2*v+1.5*v**2
    bpv[2]=0.5+v-1.5*v**2
    bpv[3]=0.5*v**2

    h=0
    dhdu=0
    dhdv=0
    for i in range(4):
        for j in range(4):
            h+= bu[i]*bv[j]*iface.r[ilat+i-2][ilong+j-2]
            dhdu+= bpu[i]*bv[j]*iface.r[ilat+i-2][ilong+j-2]
            dhdv+= bu[i]*bpv[j]*iface.r[ilat+i-2][ilong+j-2]

    dhdlat=dhdu/(h*iface.dlat0)
    dhdlong=dhdv/(h*iface.dlong0)
    norm=np.sqrt(1+dhdlat**2+dhdlong**2)

    norm_r=1/norm
    norm_lat=-dhdlat/norm
    norm_long=-dhdlong/norm

    return norm_r,norm_lat,norm_long

def interpolate_velocity():
    ir=int(np.floor((lat-gridv.lat0)/gridv.dlat0)+1) #gridv=vgrid()
    ilat=int(np.floor((lat-gridv.lat0)/gridv.dlat0)+1)
    ilong=int(np.floor((long-gridv.long0)/gridv.dlong0)+1)

    if (ir<2) or (ir>(gridv.nr-2)):
        if (ir<2):
            ir=2
        elif (ir>(gridv.nr-2)):
            ir=(gridv.nr-2)
    if (ilong<2) or (ilong>(gridv.nlong-2)):
        if (ilong<2):
            ilong=2
        elif (ilong>(gridv.nlong-2)):
            ilong=(gridv.nlong-2)
    if (ilat<2) or (ilat>(gridv.nlat-2)):
        if (ilat<2):
            ilat=2
        elif (ilat>(gridv.nlat-2)):
            ilat=(gridv.nlat-2)

    u=(lat-gridv.lat[ilat])/(gridv.dlat0)
    v=(long-gridv.long[ilong])/(gridv.dlong0)
    w=(r-gridv.r[ir])/(gridv.dr0)

    bu=np.ones(4)
    bv=np.ones(4)
    bw=np.ones(4)

    bu[0]=((1-u)**3)/6
    bu[1]=(4-6*u**2+3*u**3)/6
    bu[2]=(1+3*u+3*u**2-3*u**3)/6
    bu[3]=u**3/6

    bv[0]=((1-v)**3)/6
    bv[1]=(4-6*v**2+3*v**3)/6
    bv[2]=(1+3*v+3*v**2-3*v**3)/6
    bv[3]=v**3/6

    bw[0]=((1-w)**3)/6
    bw[1]=(4-6*w**2+3*w**3)/6
    bw[2]=(1+3*w+3*w**2-3*vw*3)/6
    bw[3]=w**3/6



    value=0
    for i in range(4):
        for j in range(4):
            for k in range(4):
                value+=bu[i]*bv[j]*bw[k]*gridv.velocity[ir+i-2][ilat+j-2][ilong+k-2]

    return value

def velocity_gradient():
    ir=np.floor((lat-gridv.lat0)/gridv.dlat0)+1 #gridv=vgrid()
    ilat=np.floor((lat-gridv.lat0)/gridv.dlat0)+1
    ilong=np.floor((long-gridv.long0)/gridv.dlong0)+1

    if (ir<2) or (ir>(gridv.nr-2)):
        if (ir<2):
            ir=2
        elif (ir>(gridv.nr-2)):
            ir=(gridv.nr-2)
    if (ilong<2) or (ilong>(gridv.nlong-2)):
        if (ilong<2):
            ilong=2
        elif (ilong>(gridv.nlong-2)):
            ilong=(gridv.nlong-2)
    if (ilat<2) or (ilat>(gridv.nlat-2)):
        if (ilat<2):
            ilat=2
        elif (ilat>(gridv.nlat-2)):
            ilat=(gridv.nlat-2)

    u=(lat-gridv.lat[ilat])/(gridv.dlat0)
    v=(long-gridv.long[ilong])/(gridv.dlong0)
    w=(r-gridv.r[ir])/(gridv.dr0)

    bu=np.ones(4)
    bv=np.ones(4)
    bw=np.ones(4)
    bpu=np.ones(4)
    bpv=np.ones(4)
    bpw=np.ones(4)

    bu[0]=((1-u)**3)/6
    bu[1]=(4-6*u**2+3*u**3)/6
    bu[2]=(1+3*u+3*u**2-3*u**3)/6
    bu[3]=u**3/6

    bv[0]=((1-v)**3)/6
    bv[1]=(4-6*v**2+3*v**3)/6
    bv[2]=(1+3*v+3*v**2-3*v**3)/6
    bv[3]=v**3/6

    bw[0]=((1-w)**3)/6
    bw[1]=(4-6*w**2+3*w**3)/6
    bw[2]=(1+3*w+3*w**2-3*vw*3)/6
    bw[3]=w**3/6

    bpu[0]=-((1-u)**2)/2
    bpu[1]=-2*u+1.5*u**2
    bpu[2]=0.5+u-1.5*u**2
    bpu[3]=0.5*u**2

    bpv[0]=-((1-v)**2)/2
    bpv[1]=-2*v+1.5*v**2
    bpv[2]=0.5+v-1.5*v**2
    bpv[3]=0.5*v**2

    bpw[0]=-((1-w)**2)/2
    bpw[1]=-2*w+1.5*w**2
    bpw[2]=0.5+w-1.5*w**2
    bpw[3]=0.5*w**2

    vv=0
    dvdu=0
    dvdv=0
    dvdw=0
    for i in range(4):
        for j in range(4):
            for k in range(4):
                vv+=bu[i]*bv[j]*bw[k]*gridv.velocity[ir+i-2][ilat+j-2][ilong+k-2]
                dvdu+=bpu[i]*bv[j]*bw[k]*gridv.velocity[ir+i-2][ilat+j-2][ilong+k-2]
                dvdv+=bu[i]*bpv[j]*bw[k]*gridv.velocity[ir+i-2][ilat+j-2][ilong+k-2]
                dvdw+=bu[i]*bv[j]*bpw[k]*gridv.velocity[ir+i-2][ilat+j-2][ilong+k-2]

    vel =vv
    dvdr=dvdu
    dvdlat=dvdv/r
    dvdlong=dvdw/(r*np.cos(lat))
    return vel,dvdr,dvdlat,dvdlong

def find_intersection(isec,iface,grid):
    isec.iface_id=iface.id # define isec/iface
    isec.pinched=iface.pinched
    isec.grid=grid #vgrid()
    
    class coordinates():
        def __init__(self):
            self.ir=0
            self.ilat=0
            self.ilong=0
    
    nodecount=grid.nr*grid.nlat*grid.nlong
    ijk_isec=[coordinates() for i in range(nodecount)]
    
    r_interface=np.zeros((grid.nlat,grid.nlong))
    r_diff=np.zeros((grid.nr,grid.nlat,grid.nlong))
    
    nextranodes=np.zeros((grid.nr+1,grid.nlat+1,grid.nlong+1))
    extranodes=np.zeros((10,grid.nr+1,grid.nlat+1,grid.nlong+1))
    type_isec=np.zeros(nodecount)
    isec.irg_bel=np.zeros((grid.nlat,grid.nlong))
    isec.irg_abo=np.zeros((grid.nlat,grid.nlong))
    
    for k in range(grid.nlong):
        for j in range(grid.nlat):
            r_interface[j][k]=interpolate_interface(grid.lat[j],grid.long[k],iface)
            for i in range(grid.nr):
                r_diff[i][j][k]=grid.r[i]-r_interface[i][j] #Doubt
    
    for k in range(grid.nlong):
        for j in range(grid.nlat):
            if (r_interface[j][k] > grid.r[grid.nr-1]):
                isec.irg_bel[j][k]=grid.nr + 1
                isec.irg_abo[j][k]=grid.nr + 1
            if (r_interface[j][k] < grid.r[1]):
                isec.irg_bel[j][k]=0
                isec.irg_abo[j][k]=0
    nodecount=0
    for k in range(grid.nlong):
        for j in range(grid.nlat):
            for i in range(grid.nr):
                if (np.abs(r_diff[i][j][k])<=grid.tolerance):
                    ijk_isec[nodecount].ir=i
                    ijk_isec[nodecount].ilat=j
                    ijk_isec[nodecount].ilong=k
                    type_isec[nodecount]=0
                    
                    nextranodes[i+1][j+1][k]+=1
                    nextranodes[i+1][j][k+1]+=1
                    nextranodes[i+1][j+1][k]+=1
                    nextranodes[i+1][j][k]+=1
                    
                    nextranodes[i][j+1][k+1]+=1
                    nextranodes[i][j][k+1]+=1
                    nextranodes[i][j+1][k]+=1
                    nextranodes[i][j][k]+=1
                    
                    extranodes[int(nextranodes[i+1][j+1][k+1])][i+1][j+1][k+1]=nodecount
                    extranodes[int(nextranodes[i+1][j][k+1])][i+1][j][k+1]=nodecount
                    extranodes[int(nextranodes[i+1][j+1][k])][i+1][j+1][k]=nodecount
                    extranodes[int(nextranodes[i+1][j][k])][i+1][j][k]=nodecount
                    
                    extranodes[int(nextranodes[i][j+1][k+1])][i][j+1][k+1]=nodecount
                    extranodes[int(nextranodes[i][j][k+1])][i][j][k+1]=nodecount
                    extranodes[int(nextranodes[i][j+1][k])][i][j+1][k]=nodecount
                    extranodes[int(nextranodes[i][j][k])][i][j][k]=nodecount
                    
                    isec.irg_abo[j][k]=i+1
                    isec.irg_bel[j][k]=i-1
                    nodecount+=1 #Doubt
                    
                else:
                    if(i<grid.nr-1):
                        if(np.sign(r_diff[i][j][k])*r_diff[i+1][j][k] < -grid.tolerance):
                            ijk_isec[nodecount].ir=i
                            ijk_isec[nodecount].ilat=j
                            ijk_isec[nodecount].ilong=k
                            type_isec[nodecount-1]=1
                            
                            nextranodes[i+1][j+1][k]+=1
                            nextranodes[i+1][j][k+1]+=1
                            nextranodes[i+1][j+1][k]+=1
                            nextranodes[i+1][j][k]+=1
                            
                            extranodes[int(nextranodes[i+1][j+1][k+1])][i+1][j+1][k+1]=nodecount
                            extranodes[int(nextranodes[i+1][j][k+1])][i+1][j][k+1]=nodecount
                            extranodes[int(nextranodes[i+1][j+1][k])][i+1][j+1][k]=nodecount
                            extranodes[int(nextranodes[i+1][j][k])][i+1][j][k]=nodecount
                            
                            isec.irg_abo[j][k]=i+1
                            isec.irg_bel[j][k]=i
                            nodecount+=1
                    if(j<grid.nlat-1):
                        if(np.sign(r_diff[i][j][k])*r_diff[i][j+1][k] < -grid.tolerance):
                            ijk_isec[nodecount].ir=i
                            ijk_isec[nodecount].ilat=j
                            ijk_isec[nodecount].ilong=k
                            type_isec[nodecount-1]=2
                            
                            nextranodes[i+1][j+1][k]+=1
                            nextranodes[i][j+1][k+1]+=1
                            nextranodes[i+1][j+1][k]+=1
                            nextranodes[i][j+1][k]+=1
                            
                            extranodes[int(nextranodes[i+1][j+1][k+1])][i+1][j+1][k+1]=nodecount
                            extranodes[int(nextranodes[i][j+1][k+1])][i][j+1][k+1]=nodecount
                            extranodes[int(nextranodes[i+1][j+1][k])][i+1][j+1][k]=nodecount
                            extranodes[int(nextranodes[i+1][j][k])][i][j+1][k]=nodecount
                            nodecount+=1
                    if(k<grid.nlong-1):
                        if(np.sign(r_diff[i][j][k])*r_diff[i][j][k+1] < -grid.tolerance):
                            ijk_isec[nodecount].ir=i
                            ijk_isec[nodecount].ilat=j
                            ijk_isec[nodecount].ilong=k
                            type_isec[nodecount-1]=3
                            
                            nextranodes[i+1][j+1][k]+=1
                            nextranodes[i+1][j][k+1]+=1
                            nextranodes[i][j+1][k+1]+=1
                            nextranodes[i][j][k]+=1
                            
                            extranodes[int(nextranodes[i+1][j+1][k+1])][i+1][j+1][k+1]=nodecount
                            extranodes[int(nextranodes[i+1][j][k+1])][i+1][j][k+1]=nodecount
                            extranodes[int(nextranodes[i][j+1][k+1])][i][j+1][k]=nodecount
                            extranodes[int(nextranodes[i][j][k+1])][i][j][k+1]=nodecount
                            nodecount+=1
    isec.nnode=nodecount
    isec.n_ccells=np.count_nonzero(nextranodes[:][:][:])
    isec.ccells=[coordinates() for i in range(isec.n_ccells)]
    isec.n_inodes=np.zeros(isec.n_ccells)
    isec.inodes=np.zeros((int(np.max(nextranodes)),isec.n_ccells))
    n=0
    for k in range(grid.nlong):
        for j in range(grid.nlat):
            for i in range(grid.nr):
                if(nextranodes[i][j][k]>0):
                    
                    isec.ccells[n].ir=i-1
                    isec.ccells[n].ilat=j-1
                    isec.ccells[n].ilong=k+1
                    
                    isec.n_inodes[n]=nextranodes[i][j][k]
                    for x in range(int(isec.n_inodes[n])):
                        isec.inodes[x][n]=extranodes[x][i][j][k]
#                     isec.inodes[:int(isec.n_inodes[n])][n]=extranodes[:int(isec.n_inodes[n])][i][j][k]
                    n+=1
                    
    counter=np.zeros(len(isec.inodes)*len(isec.inodes[0]))
    isec.ccell_from_inode=np.zeros((8,isec.nnode))
    for i in range(isec.n_ccells):
        for j in range(int(isec.n_inodes[i])):
            counter[int(isec.inodes[i][j])]+=1
            if(counter[int(isec.inodes[i][j])]>8):
                break
            isec.ccell_from_inode[int(counter[int(isec.inodes[i][j])])][int(isec.inodes[i][j])]=i
            
    for i in range(isec.nnode):
        
        if (type_isec[i]==0):
            isec.r[i]=grid.r[ijk_isec[i].ir]
            isec.lat[i]=grid.lat[ijk_isec[i].ilat]
            isec.coslat[i]=grid.coslat[ijk_isec[i].ilat]
            isec.long[i]=grid.long[ijk_isec[i].ilong]
            isec.intype[i]=type_isec[i]
        elif (type_isec[i]==1):
            isec.r[i]=r_interface[ijk_isec[i].ir].ijk_isec[i].long
            isec.lat[i]=grid.lat[ijk_isec[i].ilat]
            isec.coslat[i]=np.cos([ijk_isec[i].ilat])
            isec.long[i]=grid.long[ijk_isec[i].ilong]
            isec.intype[i]=type_isec[i]
        elif (type_isec[i]==2):
            isec.r[i]=grid.r[ijk_isec[i].ir]
            isec.lat[i]=find_zero(ijk_isec[i],type_isec[i])
            isec.coslat[i]=np.cos(isec.lat[i])
            isec.long[i]=grid.long[ijk_isec[i].ilong]
            isec.intype[i]=type_isec[i]
        elif (type_isec[i]==3):
            isec.r[i]= grid.r[ijk_isec[i].ir]
            isec.lat[i]= grid.lat[ijk_isec[i].ilat]
            isec.coslat[i]= grid.coslat[ijk_isec[i].ilat]
            isec.long[i]= find_zero(ijk_isec[n],type_isec[i])
            isec.intype[i]=type_isec[i]

def find_zero(ijk,itype):
    if(itype==2):
        x1=grid.lat[ijk.ilat] #(input grid)
        x2=grid.lat[ijk.ilat+1]
        f=rdiff(ijk.ir,ijk.ilat,ijk.ilong)
        f=rdiff(ijk.ir,ijk.ilat+1,ijk.ilong)
        if (f<0):
            rtbis=x1
            dx=x2-x1
        else:
            rtbis=x2
            dx=x1-x2
        for i in range(10):
            dx=dx*0.5
            xmid=rtbis+dx
            fmid=grid.r[ijk.ir]-interpolate_interface(xmid,grid.long[ijk.ilong],iface)
            if (fmid<=0):
                rtbis=xmid
    if(itype==3):
        x1=grid.long[ijk.ilong]
        x2=grid.long[ijk.ilong+1]
        f=rdiff(ijk.ir,ijk.ilat,ijk.ilong)
        fmid=rdiff(ijk.ir,ijk.ilat,ijk.ilong+1)
        if (f < 0):
            rtbis=x1
            dx=x2-x1
        else:
            rtbis=x2
            dx=x1-x2
        for j in range(10):
            dx=dx*0.5
            xmid=rtbis+dx
            fmid=grid.r[ijk.ir]-interpolate_interface(grid.lat[ijk.ilat],xmid,iface)
            if (fmid <= 0):
                rtbis=xmid
    return rtbis

def define_region():
    reg.ivgrid=itop.iface_id
    reg.grid = grid
    if (np.logical_and(itop.nnode > 0,ibot.nnode > 0)):
        for k in range(0,grid.nlong):
            for j in range(0,grid.nlat):
                istart=max(1,ibot.irg_abo[j][k])
                iend=min(grid.nr,itop.irg_bel[j][k])
                if (iend >= istart):
                    for i in range(istart,iend+1):
                        grid.node_region[i][j][k]=reg.id
    if (np.logical_and(itop.nnode > 0, ibot.nnode == 0)):
        for k in range(0,grid.nlong):
            for j in range(0,grid.nlat):
                for i in range(0,min(grid.nr,itop.irg_bel[j][k])):
                    grid.node_region[i][j][k] = reg.id
    if (np.logical_and(itop.nnode == 0, ibot.nnode > 0)):
        for k in range(0,grid.nlong):
            for j in range(0,grid.nlat):
                for i in range(max(1,ibot.irg_abo[j][k]),grid.nr):
                    grid.node_region[i][j][k] = reg.id
    if (np.logical_and(itop.nnode == 0,ibot.nnode == 0)):
        for k in range(0,grid.nlong):
            for j in  range(0,grid.nlat):
                for i in range(0,grid.nr):
                    grid.node_region[i][j][k] = reg.id

    itop.regbel=reg
    ibot.regabo=reg

    reg.ngnode=np.count_nonzero(grid.node_region==reg.id)
    reg.nnode=reg.ngnode+itop.nnode+ibot.nnode

    m=0
    for k in range(0,grid.nlong):
        for j in range(0,grid.nlat):
            for i in range(0,grid.nr):
                if (grid.node_region[i][j][k] == reg.id):
                    m+=1
                    reg.node[m].i1=i
                    reg.node[m].i2=j
                    reg.node[m].i3=k
                    grid.rnode_id[i][j][k]=m
                    reg.r[m] = grid.r[i]
                    reg.lat[m] = grid.lat[j]
                    reg.coslat[m] = grid.coslat[j]
                    reg.long[m] = grid.long[k]

    #m=0
    if (itop.nnode > 0):
        for i in range(0,itop.nnode):
            m+=1
            reg.node[m].i1=0
            reg.node[m].i2=itop.id
            reg.node[m].i3=i
            itop.rbel_node_id[i] = m
            reg.r[m] = itop.r[i]
            reg.lat[m] = itop.lat[i]
            reg.coslat[m] = itop.coslat[i]
            reg.long[m] = itop.long[i]
    #m=0
    if (ibot.nnode > 0):
        for i in range(0,ibot.nnode):
            m+=1
            reg.node[m].i1=0
            reg.node[m].i2=ibot.id
            reg.node[m].i3=i
            ibot.rabo_node_id[i] = m
            reg.r[m] = ibot.r[i]
            reg.lat[m] = ibot.lat[i]
            reg.coslat[m] = ibot.coslat[i]
            reg.long[m] = ibot.long[i]

def velocities_to_grid(grid):
    if(grid.is_main_grid):
        for m in range(0,n_vtypes):
            for k in range(0,grid.nlong):
                for j in range(0,grid.nlat):
                    for i in range(0,grid.nr):
                        if (grid.node_region[i][j][k]>0):
                            ivg = region(grid.node_region[i][j][k]).ivgrid
                            grid.velocity[i][j][k][m] = interpolate_velocity(grid.r[i],grid.lat[j],grid.long[k],vgrid[ivg][m])
    if (grid.is_source_grid):
        for m in range(0,n_vtypes):
            for k in range(0,grid.nlong):
                for j in range(0,grid.nlat):
                    for i in range(0,grid.nr):
                        if (grid.node_region[i][j][k]>0):
                            ivg = sregion(grid.node_region[i][j][k]).ivgrid
                            grid.velocity[i][j][k][m] = interpolate_velocity(grid.r[i],grid.lat[j],grid.long[k],vgrid(ivg,m))

def velocities_to_intersection(isec):
    vg_abo=isec.regabo.ivgrid
    for m in range(0,n_vtypes):
        for i in range(0,isec.nnode):
            isec.vel_top[i][m]=interpolate_velocity(isec.r[i],isec.lat[i],isec.long[i],vgrid(vg_abo,m))

    vg_bel = isec.regbel.ivgrid
    for m in range(0,n_vtypes):
        for i in range(0,isec.nnode):
            isec.vel_bot[i][m]=interpolate_velocity(isec.r[i],isec.lat[i],isec.long[i],vgrid(vg_bel,m))

    for i in range(0,isec.nnode):
        isig_abo = 0
        if (iface_id > 1):
            for j in range(iface_id-1,0,-1):
                h = interpolate_interface(isec.lat[i],isec.long[i],intrface[j])
                if (h-isec.r[i] > pgrid.tolerance):
                    isig_abo = j
        isig_bel = n_interfaces+1
        if (iface_id < n_interfaces):
            for j in range(iface_id+1,n_interfaces):
                h = interpolate_interface(isec.lat[i],isec.long[i],intrface(j))
                if (isec.r[i]-h > pgrid.tolerance):
                    isig_bel = j

    if (isig_bel == n_interfaces+1):
            vg_abo=region(isig_abo).ivgrid
            for n in range(0,n_vtypes):
                vel = interpolate_velocity(isec.r[i],isec.lat[i],isec.long[i],vgrid(vg_abo,n))
                if (iface_id < n_interfaces):
                    isec.vel_bot[i][n]=vel
                isec.vel_top[i][n]=vel

    vg_abo=region(isig_abo).ivgrid
    for n in range(0,n_vtypes):
        vel = interpolate_velocity(isec.r[i],isec.lat[i],isec.long[i],vgrid(vg_abo,n))
        isec.vel_top[i][n] = vel
        if (isig_bel != iface_id+1):
            isec.vel_bot[i][n] = vel

def velocities_to_region(reg,grid):
    for n in range(0,n_vtypes):
        for m in range(0,reg.nnode):
            if (reg.node[m].i1 != 0):
                reg.velocity[m][n]= grid.velocity(reg.node[m].i1,reg.node[m].i2,reg.node[m].i3,n)
            else:
                if (reg.node[m].i2 == reg.itop.id):
                    reg.velocity[m][n]= reg.itop.vel_bot(reg.node[m].i3,n)
                elif (reg.node[m].i2 != reg.ibot.id):
                    reg.velocity[m][n]= reg.ibot.vel_top(reg.node[m].i3,n)
                    break
    tag_active_vgrid_nodes(reg)

def initialize_sources(s,grid):
    s.ir    =  floor((s.r - grid.r0)/grid.dr0 + 1)
    s.ilat  =  floor((s.lat - grid.lat0)/grid.dlat0 + 1)
    s.ilong =  floor((s.long - grid.long0)/grid.dlong0 + 1)
    s.ir = min(s.ir,grid.nr-1)
    s.ilat = min(s.ilat,grid.nlat-1)
    s.ilong = min(s.ilong,grid.nlong-1)
    s.on_grid=False
    s.on_interface=False
    s.n_cnode = 0

    source_on_interface=False
    dist_below=1
    for n in range(n_interfaces,0,-1):
        dist = s.r-interpolate_interface(s.lat,s.long,intrface[n])
        source_on_interface[n] = max(abs(dist),2*grid.tolerance)
        if (source_on_interface[n]):
            print('source lies exactly on interface')
        if (np.logical_and(dist < 0, dist_below > 0)):
            s.region_id = n
    dist_below=dist

    s.on_interface = np.count((source_on_interface[1:n_interfaces]) > 0)
    s.on_pinched_interface = np.count((source_on_interface[1:n_interfaces]) > 1)

    if (s.on_interface):
        s.topint_id = 0
        s.botint_id = n_interfaces+1
        for i in range(0,n_interfaces):
            h = interpolate_interface(s.lat,s.long,intrface[i])
            if (h-s.r > pgrid.tolerance): s.topint_id = i

        for i in range(n_interfaces,1,-1):
            h = interpolate_interface(s.lat,s.long,intrface[i])
            if (s.r-h > pgrid.tolerance): s.botint_id = i

        s.topint_id=s.topint_id+1
        s.botint_id=s.botint_id-1

        s.topreg_id=s.topint_id-1
        s.botreg_id=s.botint_id

        if (np.logical_or(s.topint_id == 1, s.botint_id == n_interfaces)):
            s.n_tf_init = 1
        else:
            s.n_tf_init = 2

    if (~s.on_interface):
        for i in range(0,1):
            for j in range(0,1):
                for k in range(0,1):
                    dist=np.sqrt((s.r-grid.r[s.ir+i])**2 + (s.lat-grid.lat[s.ilat+j])**2 + (s.long-grid.long[s.ilong+k])**2 )
                if (dist < grid.tolerance): s.on_grid=True
    s.n_tf_init = 1

def sweep_region_from_interface(reg,istart_in,vtype,s):
    istart=istart_in
    itop=reg.itop
    ibot=reg.ibot

    received_turning_ray= False
    reg.node_status=-1

    for i in range(0,istart.nnode):
        istart.starttime[i]=istart.arrivaltime[i]
        if (istart.id == ibot.id):
            j=istart.rabo_node_id[i]
        else:
            j=istart.rbel_node_id[i]
        reg.arrivaltime[j]= istart.arrivaltime[i]
        reg.time_gradient[:3][j]=istart.time_gradient[:3][i]
        reg.node_status[j]=1

    propagate(reg,vtype)

    n_turning=0

    for i in range(0,reg.nnode):
        i1 = reg.node[i].i1
        i2 = reg.node[i].i2
        i3 = reg.node[i].i3
        if (i1==0):
            if (i2 == itop.id):
                itop.arrivaltime[i3] = reg.arrivaltime[i]
                itop.time_gradient[:3][i3] = reg.time_gradient[:3][i]
            else:
                ibot.arrivaltime[i3] = reg.arrivaltime[i]
                ibot.time_gradient[:3][i3] = reg.time_gradient[:3][i]
        if (i2 == istart.id):
            if (istart.arrivaltime[i3] < istart.starttime[i3]):
                n_turning+=1
                received_turning_ray[i3] = True

    if (n_turning > 0):
        print(n_turning,'starting intersection nodes received a turning ray')

    s.n_time_fields+=1
    s.time_field(s.n_time_fields).arrivaltime=reg.arrivaltime

    tg=reg.time_gradient
    s.time_field(s.n_time_fields).time_gradient=tg

    if (n_turning > 0):
        s.time_field(s.n_time_fields).turning_rays_present = True
        s.time_field(s.n_time_fields).received_turning_ray = received_turning_ray

def sweep_sregion_from_interface(reg,istart_in,vtype):
    istart = istart_in
    itop = reg.itop
    ibot = reg.ibot

    reg.node_status=-1

    for i in range(0,istart.nnode):
        istart.starttime[i]=istart.arrivaltime[i]
        if (istart.id == ibot.id):
            j=istart.rabo_node_id[i]
        else:
            j=istart.rbel_node_id[i]

    reg.arrivaltime[j]= istart.arrivaltime[i]
    reg.time_gradient[:3][j]=istart.time_gradient[:3][i]
    reg.node_status[j]=1

    propagate(reg,vtype)

def sweep_region_from_source(reg,s,vtype):
    grid=reg.grid

    itop=reg.itop
    ibot=reg.ibot

    vel_source=interpolate_velocity(s.r,s.lat,s.long,vgrid(reg.ivgrid,vtype))

    reg.node_status=-1
    reg.arrival_time=huge_time

    if (s.n_cnode == 1):  # source lies exactly on a node, only 1 starting point with time 0
        i1=s.cnode[1].i1
        i2=s.cnode[1].i2
        i3=s.cnode[1].i3
        if (i1 != 0):     # node is a regular grid point
            j= grid.rnode_id(i1,i2,i3)
            reg.arrivaltime[j]= 0
            reg.time_gradient[:3][j]= 0
            reg.node_status[j]=1
        else:             # node is an interface node
            if (i2 == ibot.id):
                j=ibot.rabo_node_id[i3]
            else:
                j=itop.rbel_node_id[i3]
            reg.arrivaltime[j]= 0
            reg.time_gradient[:3][j]= 0
            reg.node_status[j]= 1
            print('source node',j,'set to zero time')

    else:
        for m in  range(0,s.n_cnode):
            i1=s.cnode[m].i1
            i2=s.cnode[m].i2
            i3=s.cnode[m].i3
            if (i1 != 0):  # node is a regular grid point
                j= grid.rnode_id[i1][i2][i3]
                vel_av=0.5*(grid.velocity[i1][i2][i3][vtype]+vel_source)
                dist=((s.r-grid.r[i1])**2 + (s.r*(s.lat-grid.lat[i2]))**2 + (s.r*s.coslat*(s.long-grid.long[i3]))**2 )
                reg.arrivaltime[j]= dist/vel_av
                reg.time_gradient[0][j]= (grid.r[i1]-s.r)/(dist*grid.velocity[i1][i2][i3][vtype])
                reg.time_gradient[1][j]= s.r*(grid.lat[i2]-s.lat)/(dist*grid.velocity[i1][i2][i3][vtype])
                reg.time_gradient[2][j]= s.r*s.coslat*(grid.long[i3]-s.long)/(dist*grid.velocity[i1][i2][i3][vtype])
                reg.node_status[j]=1
            else:
                if (i2 == ibot.id):   # node lies on the bottom interface of the region
                    j=ibot.rabo_node_id[i3]
                    dist=((s.r-ibot.r[i3])**2 + (s.r*(s.lat-ibot.lat[i3]))**2 + (s.r*s.coslat*(s.long-ibot.long[i3]))**2 )
                    reg.arrivaltime[j]= dist/ibot.vel_top[i3][vtype]
                    reg.time_gradient[0][j]= (ibot.r[i3]-s.r)/(dist*ibot.vel_top[i3][vtype])
                    reg.time_gradient[1][j]= s.r*(ibot.lat[i3]-s.lat)/(dist*ibot.vel_top[i3][vtype])
                    reg.time_gradient[2][j]= s.r*s.coslat*(ibot.long[i3]-s.long)/(dist*ibot.vel_top[i3][vtype])
                else:                     # node lies on the top interface of the region
                    j=itop.rbel_node_id[i3]
                    dist=np.sqrt((s.r-itop.r[i3])**2 + (s.r*(s.lat-itop.lat[i3]))**2 + (s.r*s.coslat*(s.long-itop.long[i3]))**2 )
                    reg.arrivaltime[j]= dist/itop.vel_bot[i3][vtype]
                    reg.time_gradient[0][j]= (itop.r[i3]-s.r)/(dist*itop.vel_bot[i3][vtype])
                    reg.time_gradient[1][j]= s.r*(itop.lat[i3]-s.lat)/(dist*itop.vel_bot[i3][vtype])
                    reg.time_gradient[2][j]= s.r*s.coslat*(itop.long[i3]-s.long)/(dist*itop.vel_bot[i3][vtype])
            reg.node_status[j]=1
    propagate(reg,vtype)

def initialise_refined_sources(s,sc,grid,reg,itop,ibot):
    s.r=sc.r
    s.lat=sc.lat
    s.long=sc.long
    s.coslat=sc.coslat
    s.region_id=reg.id

    s.ir    =  np.floor((s.r - grid.r0)/grid.dr0 + 1)
    s.ilat  =  np.floor((s.lat - grid.lat0)/grid.dlat0 + 1)
    s.ilong =  np.floor((s.long - grid.long0)/grid.dlong0 + 1)

    s.ir = min(s.ir,grid.nr-1)
    s.ilat = min(s.ilat,grid.nlat-1)
    s.ilong = min(s.ilong,grid.nlong-1)

    s.on_grid=False
    s.on_interface=sc.on_interface
    s.n_cnode = 0

    if (s.on_interface):
        if (sc.topint_id == ibot.id ):
            s.interface_id = ibot.id
            isec = ibot
        elif (sc.botint_id == itop.id):
            s.interface_id = itop.id
            isec = itop

    iface=isec.iface_id

    # test if the source lies exactly on an intersection node

    for m in range(0,isec.nnode):
        dist =((s.r-isec.r(m))**2 +(s.r*(s.lat-isec.lat(m)))**2 + (s.r*s.coslat*(s.long-isec.long(m)))**2 )
        if (abs(dist) < grid.tolerance):  # source lies exactly on an interface node
            # assign time value 0 to the source node
            s.n_cnode = s.n_cnode + 1
            s.cnode[s.n_cnode].i1=0
            s.cnode[s.n_cnode].i2=isec.id
            s.cnode[s.n_cnode].i3=m
            s.on_grid=True
            print('source lies on node ',m,' of intersection ',isec.id)

    if (~s.on_grid):   # source lies on interface but not on an interface node
        print('source lies exactly on interface',iface,'but not on a node')
        #  find the nodes of intersection that are part of the cell containing the source
        m= grid.ccind_from_3dc[s.ir][s.ilat][s.ilong].p[iface]
        for i in range(0,isec.n_inodes[m]):
            k=isec.inodes[i][m]
            s.n_cnode+=1
            s.cnode[s.n_cnode].i1=0
            s.cnode[s.n_cnode].i2=isec.id
            s.cnode[s.n_cnode].i3=k
        # the regular nodes of the cut cell containing the source
        for i in range(0,1):
            is_=s.ir+i
            for j in range(0,1):
                js=s.ilat+j
                for k in range(0,1):
                    ks=s.ilong+k
                    if (np.logical_and(is_ > 0, is_ <= grid.nr,js > 0, js <= grid.nlat, ks > 0, ks <= grid.nlong, grid.node_region[is_][js][ks] == s.region_id)):
                        s.n_cnode = s.n_cnode + 1
                        s.cnode[s.n_cnode].i1=is_
                        s.cnode[s.n_cnode].i2=js
                        s.cnode[s.n_cnode].i3=ks

    if (~s.on_interface):
        for i in range(0,1):
            for j in range(0,1):
                for k in range(0,1):
                    dist=np.sqrt((s.r-grid.r[s.ir+i])**2 + (s.r*(s.lat-grid.lat[s.ilat+j]))**2 + (s.r*s.coslat*(s.long-grid.long[s.ilong+k]))**2 )
                    if (dist < grid.tolerance):
                        s.on_grid=True
                        s.n_cnode += 1
                        s.cnode[s.n_cnode].i1=s.ir+i
                        s.cnode[s.n_cnode].i2=s.ilat+j
                        s.cnode[s.n_cnode].i3=s.ilong+k
                        print('source on grid but not interface')

    if (~s.on_grid):
        print('source does not lie on grid')
        # test if the cell in which the source resides is cut
        if (associated(grid.ccind_from_3dc[s.ir][s.ilat][s.ilong].p)):
            print('source lies in a cut cell')        # if so, make a list of the nodes in the cut cell
            # first the intersection nodes

            for n in range(1,2):
                if (n == 1):
                    isec = itop
                if (n == 2):
                    isec = ibot
                if (grid.ccind_from_3dc[s.ir][s.ilat][s.ilong].p[isec.iface_id] != 0) :
                    m= grid.ccind_from_3dc[s.ir][s.ilat][s.ilong].p[isec.iface_id]
                    for i in range(0,isec.n_inodes[m]):
                        k=isec.inodes[i][m]
                        s.n_cnode += 1
                        s.cnode[s.n_cnode].i1=0
                        s.cnode[s.n_cnode].i2=isec.id
                        s.cnode[s.n_cnode].i3=k

        # then the regular nodes

            for i in range(-1,2):
                is_=s.ir+i
                for j in range(-1,2):
                    js=s.ilat+j
                    for k in range(-1,2):
                        ks=s.ilong+k
                        if (np.logical_and(is_ > 0, is_ <= grid.nr, js > 0, js <= grid.nlat, ks > 0, ks <= grid.nlong, grid.node_region[is_][js][ks] == s.region_id)):
                            s.n_cnode+=1
                            s.cnode[s.n_cnode].i1=is_
                            s.cnode[s.n_cnode].i2=js
                            s.cnode[s.n_cnode].i3=ks

        else:
            # the cell in which the source lies is not cut
            # make a list of the nodes in the regular cell
            print('source lies in an uncut cell')
            for i in range(-1,2):
                is_ =s.ir+i
                for j in range(-1,2):
                    js=s.ilat+j
                    for k in range(-1,2):
                        ks=s.ilong+k
                        if (np.logical_and(is_> 0, is_ <= grid.nr, js > 0, js <= grid.nlat, ks > 0, ks <= grid.nlong, grid.node_region[is_][js][ks] == s.region_id)):
                            s.n_cnode+=1
                            s.cnode(s.n_cnode).i1=is_
                            s.cnode(s.n_cnode).i2=js
                            s.cnode(s.n_cnode).i3=ks

def initialise_source_regions(s):
    pgrid_defaults(sgrid)
    sgrid.is_source_grid = True
    # limits of volume of main grid to be refined, taking main grid boundaries into account
    i =  nint((s.r - pgrid.r0)/pgrid.dr0 + 1)
    j =  nint((s.lat - pgrid.lat0)/pgrid.dlat0 + 1)
    k =  nint((s.long - pgrid.long0)/pgrid.dlong0 + 1)

    nrmax = min(pgrid.nr , i + ncell_to_be_refined)
    nrmin = max( 1 , i - ncell_to_be_refined)
    nlatmax = min(pgrid.nlat , j + ncell_to_be_refined)
    nlatmin = max( 1 , j - ncell_to_be_refined)
    nlongmax = min(pgrid.nlong , k + ncell_to_be_refined)
    nlongmin = max( 1 , k - ncell_to_be_refined)

    # origin of the refined source grid in propagation grid index coordinates

    sgrid.index_r0 = nrmin
    sgrid.index_lat0 = nlatmin
    sgrid.index_long0 = nlongmin

    # calculate the refined source grid parameters

    sgrid.nr      = (nrmax - nrmin)*refinement_factor + 1
    sgrid.nlat    = (nlatmax - nlatmin)*refinement_factor + 1
    sgrid.nlong   = (nlongmax - nlongmin)*refinement_factor + 1
    sgrid.dr0     = pgrid.dr0/refinement_factor
    sgrid.dlat0   = pgrid.dlat0/refinement_factor
    sgrid.dlong0  = pgrid.dlong0/refinement_factor
    sgrid.r0      = pgrid.r[s.ir] - (s.ir - nrmin)*pgrid.dr0
    sgrid.lat0    = pgrid.lat[s.ilat] - (s.ilat - nlatmin)*pgrid.dlat0
    sgrid.long0   = pgrid.long[s.ilong] - (s.ilong - nlongmin)*pgrid.dlong0


    # initialize the grid coordinates
    for i in range(0,sgrid.nr):
        sgrid.r[i]=sgrid.r0 + (i-1)*sgrid.dr0

    for i in range(0,sgrid.nlat):
        sgrid.lat[i]=sgrid.lat0 + (i-1)*sgrid.dlat0

    for i in range(0,sgrid.nlat):
        sgrid.coslat[i]=cos(sgrid.lat[i])

    for i in range(0,sgrid.nlong):
        sgrid.long[i]=sgrid.long0 + (i-1)*sgrid.dlong0

    sgrid.tolerance = refinement_factor*interface_tolerance*sgrid.dr0
    # allocate storage for the refined source grid, intersections and regions


    sgrid.rnode_id=0
    sgrid.node_region=0
    sgrid.arrivaltime = huge_time

    n_sintersections=n_intersections

    for i in range(0,n_sintersections):
        intersection_defaults(sintersection[i])
        sintersection[i].id=i

    n_sregions=n_regions

    for i in range(0,n_sregions):
        region_defaults(sregion[i])
        sregion[i].id=i

    for n in range(0,n_sintersections):
        find_intersection(sintersection[n],intrface[n],sgrid)
        print('intersections found')
    # set a flag at nodes on the grid that are completely regular, i.e. none of the connected cells
    # has irregular nodes

    tag_regular_nodes(sgrid)

    print('nodes of refined source grid tagged')
    for n in range(0,n_sregions):
        sregion[n].id=n
        # pointers to the intersections that are the boundaries of the regions
        region[n].itop = sintersection[n]
        sregion[n].ibot = sintersection[n+1]
        sregion[n].ivgrid=region[n].ivgrid

        if (np.logical_and(sregion[n].itop.nnode == 0, sregion[n].ibot.nnode == 0, sregion[n].id != s.region_id)):
            print('sregion',n,'does not exist in refined source grid')
        sregion[n].nnode = 0
        define_region(sregion[n],sintersection[n],sintersection(n+1),sgrid)
        print('refined source region',n,' defined, nnode =',sregion[n].nnode)

        # transfer the velocity values to all refined grid nodes

    velocities_to_grid(sgrid)

    for n in range(0,n_sintersections):
        if (sintersection[n].nnode != 0):
            velocities_to_intersection(sintersection[n])

    for n in range(0,n_sregions):
        if (sregion[n].nnode > 0):
            velocities_to_region(sregion[n],sgrid)

    print('refined velocities transferred')

    #------------------------------------------------------------------------------------------------
    # determine the paths up and down from the source region covering the main regions overlapping
    # with the refined source region
    #------------------------------------------------------------------------------------------------

    # set default values for the intialization paths

    for i in range(1,2):
        for j in range(1,2):
            path_defaults(s.init_path[i][j])
            s.init_path[i][j].used = False

            # first construct the path up if it exists

    for vtype in range(0,n_vtypes):
        # # first construct the path up if it exists ( init_path[0][vtype] )
        if (~np.logical_and(s.on_interface, s.topint_id == 1)):
            #   print *,'constructing source sequence up'
            if (s.on_interface):
                nstart = s.topreg_id - 1
            else:
                nstart = s.region_id - 1

        s.init_path[0][vtype].n_tf = 1
        for n in range(nstart,1,-1):     # count the number of timefields up
            if (sregion[n].nnode > 0):
                s.init_path[1][vtype].n_tf+=1
        s.init_path[0][vtype].id  = 0
        s.init_path[0][vtype].valid= True
        s.init_path[0][vtype].used = True
        s.init_path[0][vtype].refstep = 0
        s.init_path[0][vtype].fitting_interface = 0

        s.init_path[0][vtype].sequence[1] = 0
        if (s.on_interface):
            s.init_path[0][vtype].sequence[2] = region[s.topreg_id].itop.iface_id
        else:
            s.init_path[0][vtype].sequence[2] = region[s.region_id].itop.iface_id


        # construct the sequence of the first path
        if (s.init_path[0][vtype].n_tf > 1):
            for i in range(1,s.init_path[0][vtype].n_tf):
                s.init_path[0][vtype].sequence[2*i-1] = s.init_path[0][vtype].sequence[2*i-2]
                s.init_path[0][vtype].sequence[2*i] = s.init_path[0][vtype].sequence[2*i-1] - 1
        # # then construct the path down if it exists ( init_path(2,vtype) )

        if (~np.logical_and(s.on_interface, s.botint_id == n_interfaces)):
            if (s.on_interface):
                nstart = s.botreg_id + 1
            else:
                nstart = s.region_id + 1
            s.init_path(2,vtype).n_tf = 1
            for n in range(nstart,n_sregions):     # count the number of timefields down
                if (sregion[n].nnode > 0):
                    s.init_path(2,vtype).n_tf+=1

            s.init_path[2][vtype].id  = 0
            s.init_path[2][vtype].valid= True
            s.init_path[2][vtype].used = True
            s.init_path[2][vtype].refstep = 0
            s.init_path[2][vtype].fitting_interface = 0

            s.init_path[2][vtype].sequence[1] = 0
            if(s.on_interface):
                s.init_path[2][vtype].sequence[2] = region[s.botreg_id].ibot.iface_id
            else:
                s.init_path[2][vtype].sequence[2] = region[s.region_id].ibot.iface_id

            # construct the sequence of the second path
            if (s.init_path[2][vtype].n_tf > 1):
                for i in range(2,s.init_path[2][vtype].n_tf):
                    s.init_path[2][vtype].sequence[2*i-1] = s.init_path[2][vtype].sequence[2*i-2]
                    s.init_path[2][vtype].sequence[2*i] = s.init_path[2][vtype].sequence[2*i-1] + 1

    for vtype in range(1,n_vtypes):
        print('--------------------------------------------------')
        print('starting source initialization for vtype =',vtype)
        # for the propagation through the source regions first

        # if the source lies on an interface initialize both regions above and below, else only source region

        for n in range(1,s.n_tf_init):
            if (s.on_interface):   # initialize the regions above and below the interface
                if (s.n_tf_init == 2):
                    if (n==1):
                        sreg = sregion[s.topreg_id]
                    if (n==2):
                        sreg = sregion[s.botreg_id]
                else:
                    if (s.topreg_id > 0):
                        sreg = sregion[s.topreg_id]
                    if (s.botreg_id < n_regions):
                        sreg = sregion[s.botreg_id]
            else:                       # only he one region in which the source lies
                sreg = sregion[s.region_id]

            itop = sintersection[sreg.id]     # top intersection of the refined source region
            ibot = sintersection[sreg.id+1]   # bottom intersection of the refined source region


            # initialize the source in the refined source grid.

            initialize_refined_source(ss,s,sgrid,sreg,itop,ibot)

            print('refined source initialized in sregion',sreg.id)

            # sweep the refined source grid

            sweep_region_from_source(sreg,ss,vtype)
            print('refined source region',sreg.id,' swept')

            # transfer regional travel times to interfaces and regular grid
            #  if (itop.nnode > 0 .and. .not. associated(itop.arrivaltime)) allocate(itop.arrivaltime(itop.nnode))
            #  if (ibot.nnode > 0 .and. .not. associated(ibot.arrivaltime)) allocate(ibot.arrivaltime(ibot.nnode))


            for i in range(1,sreg.nnode):
                i1 = sreg.node[i].i1
                i2 = sreg.node[i].i2
                i3 = sreg.node[i].i3

                if (i1 == 0):
                    sintersection[i2].arrivaltime[i3] = sreg.arrivaltime[i]
                    sintersection[i2].time_gradient[:3][i3] = sreg.time_gradient[:3][i]
                else:
                    sgrid.arrivaltime[i1][i2][i3]= sreg.arrivaltime[i]

    #--------------------------------------------------------------------------------
    # now sweep up and down through the remaining regions of the refined source grid

    # sweep up

    if (np.logical_or((~s.on_interface), np.logical_and(s.on_interface, s.topreg_id > 1))):
        if (s.on_interface):
            nstart = s.topreg_id - 1
        else:
            nstart = s.region_id - 1

        for n in range(nstart,1,-1):
            sreg = sregion[n]

            if (sreg.nnode > 0):
                refract_gradient(sintersection[n+1],sregion[sreg.id+1],vtype,1)
                sweep_sregion_from_interface(sreg,sintersection[n+1],vtype)

                print('refined region',n,' swept')

                # transfer regional travel times to interfaces

                for i in range(1,sreg.nnode):
                    i1 = sreg.node[i].i1
                    i2 = sreg.node[i].i2
                    i3 = sreg.node[i].i3

                    if (i1 == 0):
                        sintersection[i2].arrivaltime[i3] = sreg.arrivaltime[i]
                        sintersection[i2].time_gradient[:3][i3] = sreg.time_gradient[:3][i]
                    else:
                        sgrid.arrivaltime[i1][i2][i3]= sreg.arrivaltime[i]

    # sweep down

    if (np.logical_or(~s.on_interface, np.logical_and(s.on_interface, s.botreg_id < n_regions))):
        if (s.on_interface):
            nstart = s.botreg_id + 1
        else:
            nstart = s.region_id + 1

        for n in range(nstart,n_sregions):
            sreg = sregion[n]

            if (sreg.nnode > 0):

                print('refracting at interface',n)
                refract_gradient(sintersection[n],sregion[sreg.id-1],vtype,-1)
                print('starting sweep from interface',n,'into region',sreg.id)
                sweep_sregion_from_interface(sreg,sintersection[n],vtype)

                print('refined region',n,' swept')

                for i in range(1,sreg.nnode):
                    i1 = sreg.node[i].i1
                    i2 = sreg.node[i].i2
                    i3 = sreg.node[i].i3

                    if (i1 == 0):
                        #               write(1,*) i,i1,i2,i3
                        sintersection[i2].arrivaltime[i3] = sreg.arrivaltime[i]
                        sintersection[i2].time_gradient[:3][i3] = sreg.time_gradient[:3][i]
                    else:
                        #               write(1,*) i,i1,i2,i3

                        sgrid.arrivaltime[i1][i2][i3]= sreg.arrivaltime[i]
    # call system_clock(t2)
    print('refined grid for source',s.id,'took', real(t2-t1)/10000.,' sec')

    # transfer the results to the region(s) in which the source resides on the main propagation grid
    # first find shortest traveltime on refined region boundary
    t_short=huge_time
    rmin=pgrid.r[1]+pgrid.tolerance
    rmax=pgrid.r[pgrid.nr]-pgrid.tolerance
    latmin=pgrid.lat[1]+pgrid.tolerance/pgrid.dr0
    latmax=pgrid.lat[pgrid.nlat]-pgrid.tolerance/pgrid.dr0
    longmin=pgrid.long[1]+pgrid.tolerance/pgrid.dr0
    longmax=pgrid.long[pgrid.nlong]-pgrid.tolerance/pgrid.dr0

    if (sgrid.r[1] > rmin):
        t_short = min(t_short,np.min(sgrid.arrivaltime[1][1:sgrid.nlat][1:sgrid.nlong]))
    if (sgrid.r[sgrid.nr] < rmax):
        t_short = min(t_short,np.min(sgrid.arrivaltime[sgrid.nr][1:sgrid.nlat][1:sgrid.nlong]))
    if (sgrid.lat[1] > latmin):
        t_short = min(t_short,np.min(sgrid.arrivaltime[1:sgrid.nr][1][1:sgrid.nlong]))
    if (sgrid.lat[sgrid.nlat] < latmax):
        t_short = min(t_short,np.min(sgrid.arrivaltime[1:sgrid.nr][sgrid.nlat][1:sgrid.nlong]))
    if (sgrid.long[1] > longmin):
        t_short = min(t_short,np.min(sgrid.arrivaltime[1:sgrid.nr][1:sgrid.nlat][1]))
    if (sgrid.long[sgrid.nlong] < longmax):
        t_short = min(t_short,np.min(sgrid.arrivaltime[1:sgrid.nr][1:sgrid.nlat][sgrid.nlong]))

    print('shortest time on refined grid boundary is',t_short)
    # transfer the refined source region(s) to the main propagation grid source region(s)
    # if the source lies on an interface initialize both regions above and below, else only source region

    for n in range(2):
        if (s.init_path[n][vtype].used):
            if (s.on_interface):   # initialize the regions above and below the interface
                if (n==1):
                    sreg = sregion[s.topreg_id]
                    reg = region[s.topreg_id]
                if (n==2):
                    sreg = sregion[s.botreg_id]
                    reg = region[s.botreg_id]
            else:                  # only the one region in which the source lies
                sreg = sregion[s.region_id]
                reg = region[s.region_id]

            itopc = intersection[reg.id]      # the top intersection of the normal source region
            ibotc = intersection[reg.id+1]    # the bottom intersection of the normal source region
            reg.node_status=-1
            reg.arrivaltime = huge_time

            print('before transfer refined region')
            transfer_refined_region(sreg,reg,t_short)
            print('transferred times from fine grid to coarse for region',reg.id)

            # create a narrow band around the nodes transferred from the fine grid

            create_narrow_band(reg,vtype)

            print('narrow band created in region',reg.id,'alive/nb',np.count_nonzero(reg.node_status == 0),np.count_nonzero(reg.node_status == 1))
            # for the fast marching sweep across the main grid region containing the source
            propagate(reg,vtype)
            print('propagation through region',reg.id,' finished')
            # transfer regional travel times to interfaces
            for i in range(1,reg.nnode):
                i1 = reg.node[i].i1
                i2 = reg.node[i].i2
                i3 = reg.node[i].i3
                if (i1 == 0):
                    intersection[i2].arrivaltime[i3] = reg.arrivaltime[i]
                    intersection[i2].time_gradient[1:3][i3] = reg.time_gradient[1:3][i]
                    # transfer time field to the array of saved time fields

            s.n_time_fields+=1
            s.time_field[s.n_time_fields].arrivaltime=reg.arrivaltime
            tg=reg.time_gradient
            s.time_field[s.n_time_fields].time_gradient=tg
            print('results written to timefield',s.n_time_fields)

            # pointer to source region
            s.time_field[s.n_time_fields].reg =  reg
            s.time_field[s.n_time_fields].vtype =  vtype
            s.init_path[n][vtype].tf_sequence[1] = s.n_time_fields

            if (s.on_interface):
                if (reg.id == s.topreg_id):
                    s.time_field[s.n_time_fields].istart = intersection[s.topint_id]
                    s.time_field[s.n_time_fields].inonstart =  intersection[s.topint_id - 1]
                else:
                    s.time_field[s.n_time_fields].istart = intersection[s.botint_id]
                    s.time_field[s.n_time_fields].inonstart =  intersection[s.botint_id + 1]
            else:
                s.time_field[s.n_time_fields].istart = ibotc
                s.time_field[s.n_time_fields].inonstart =  itopc

            # register the first (source) time fields with the associated source

            if (n == 1):
                s.first_tf_up[vtype] = s.n_time_fields
            if (n == 2):
                s.first_tf_down[vtype] = s.n_time_fields

            # if the source does not lie on an interface, the first timefields up and down are the same
            # we for not need to for the direction down (loop index n = 2)

            if (~s.on_interface):
                if (n==2):
                    print('error n==2 in source region init')
                    break
                    s.init_path[2][vtype].tf_sequence[1] = s.n_time_fields
                    s.first_tf_down[vtype] = s.n_time_fields

            # timefields 1(/2) describing the source region(s) on the main grid ha(s)(ve) been established
    print('timefields in main grid regions connected to the source established, vtype=',vtype)

    # now construct the actual time field sequences on the main grid up (path 1) or down (path 2)
    # through the regions overlapping with the refined source grid
    # the upward path 1

    # only if there are regions above the source region

    if (np.logical_and(s.init_path[0][vtype].used, s.init_path[0][vtype].n_tf > 1)):
        prev_tf = s.first_tf_up[vtype]
        print('starting upward sweep in main grid regions overlapping source region')

        for n in range(2,s.init_path[0][vtype].n_tf):
            # transfer the refined source region to the main propagation grid region
            if (s.on_interface):
                reg = region[s.topreg_id-n+1]
            else:
                reg =  region[s.region_id-n+1]
                # the region in the main propagation grid
            itopc = reg.itop                       # the top intersection of the region
            ibotc = reg.ibot                       # the bottom intersection of the region
            reg.node_status=-1
            reg.arrivaltime = huge_time

            transfer_refined_region(sregion[reg.id],reg,t_short)
            print('transferred times from fine grid to coarse for region',reg.id)
            # create narrow band around transferred nodes

            create_narrow_band(reg,vtype)
            print('narrow band created in region',reg.id,'alive/nb',np.count_nonzero(reg.node_status == 0),np.count_nonzero(reg.node_status == 1))

            # add the intersection nodes to the narrow band

            if (~associated(ibotc.arrivaltime)):
                print('requested starting interface does not have arrival times')

                # start time will be compared to arrival time after propagation to find turning rays
                # set start time, time gradient and node status narrow band in regional
                # nodes that belong to the starting interface
            refract_gradient(ibotc,region[ibotc.iface_id],vtype,1)
            for i in range(ibotc.nnode):
                j=ibotc.rabo_node_id[i]
                if (reg.node_status[j] != 0):
                    # only for nodes that did not receive a value from the refined grid
                    reg.arrivaltime[j]= ibotc.arrivaltime[i]
                    reg.time_gradient[:3][j]=ibotc.time_gradient[:3][i]
                    reg.node_status[j]=1
                ibotc.starttime[i]=reg.arrivaltime[j]

                # for the fast marching sweep across the main grid region
            propagate(reg,vtype)
            print('propagation through region',reg.id,' finished')

            # transfer regional travel times to interfaces

            for i in range(1,reg.nnode):
                i1 = reg.node[i].i1
                i2 = reg.node[i].i2
                i3 = reg.node[i].i3
                if (i1 == 0):
                    intersection[i2].arrivaltime[i3] = reg.arrivaltime[i]
                    intersection[i2].time_gradient[:3][i3] = reg.time_gradient[:3][i]

                # transfer time field to the array of saved time fields

            s.n_time_fields+=1
            s.time_field[s.n_time_fields].arrivaltime=reg.arrivaltime
            tg=reg.time_gradient
            s.time_field[s.n_time_fields].time_gradient=tg

            print('results written to timefield',s.n_time_fields)
            # attach info to the generated time field
            # time field preceding the current one

            s.time_field[s.n_time_fields].prev_tf = prev_tf

            # identify the current time field as the child of the previous time field
            if (prev_tf <= s.n_tf_init):
                s.time_field[prev_tf].next_tf[1+(vtype-1)*4] = s.n_time_fields
            else:
                s.time_field[prev_tf].next_tf[1+(vtype-1)*4] = s.n_time_fields
            # store the index of the time field in the sequence of timefields for this path
            s.init_path[0][vtype].tf_sequence[n] = s.n_time_fields

            # pointers to start and non-start interfaces
            s.time_field[s.n_time_fields].istart = ibotc
            s.time_field[s.n_time_fields].inonstart =  itopc
            s.time_field[s.n_time_fields].reg =  reg

            s.time_field[s.n_time_fields].vtype =  vtype
            prev_tf = s.n_time_fields

            # check for turning rays

            if (np.count_nonzero(ibotc.arrivaltime != ibotc.starttime) > 0):
                print('turning rays were present on interface',ibotc.iface_id)
                s.time_field[s.n_time_fields].turning_rays_present=True
                s.time_field[s.n_time_fields].received_turning_ray = (ibotc.arrivaltime != ibotc.starttime)

    if (np.logical_and(s.init_path[2][vtype].used, s.init_path[2][vtype].n_tf > 1)):
        prev_tf =s.first_tf_down[vtype]
        print('starting downward sweep in main grid regions overlapping source region')
        for n in range(2,s.init_path[2][vtype].n_tf):
            print('timefield',n,' of the downward sequence started')
            # transfer the refined source region to the main propagation grid region
            if (s.on_interface):
                reg = region[s.botreg_id+n-1]
            else:
                reg =  region[s.region_id+n-1]         # the region in the main propagation grid

            itopc = reg.itop                       # the top intersection of the region
            ibotc = reg.ibot                       # the bottom intersection of the region
            reg.node_status=-1
            reg.arrivaltime = huge_time

            transfer_refined_region(sregion[reg.id],reg,t_short)
            print('transferred times from fine grid to coarse for region',reg.id)

            # create narrow band around transferred nodes

            create_narrow_band(reg,vtype)
            print('narrow band created in region',reg.id,'alive/nb',np.count_nonzero(reg.node_status == 0),np.count_nonzero(reg.node_status == 1))
            # add the intersection nodes to the narrow band
            if (~associated(itopc.arrivaltime)):
                print('requested starting interface does not have arrival times')

                # set start time, time gradient and node status narrow band in regional
                # nodes that belong to the starting interface
            refract_gradient(itopc,region[itopc.iface_id-1],vtype,-1)

            for i in range(1,itopc.nnode):
                j=itopc.rbel_node_id[i]
                if (reg.node_status[j] != 0):
                    # only for nodes that did not receive a value from the refined grid
                    reg.arrivaltime[j]= itopc.arrivaltime[i]
                    reg.time_gradient[:3][j]=itopc.time_gradient[:3][i]
                    reg.node_status[j]=1

                    itopc.starttime[i]=reg.arrivaltime[j]

            # for the fast marching sweep across the main grid region

            propagate(reg,vtype)
            print('propagation through region',reg.id,' finished')

            # transfer regional travel times to interfaces

            for i in range(1,reg.nnode):
                i1 = reg.node[i].i1
                i2 = reg.node[i].i2
                i3 = reg.node[i].i3
                if (i1 == 0):
                    intersection[i2].arrivaltime[i3] = reg.arrivaltime[i]
                    intersection[i2].time_gradient[:3][i3] = reg.time_gradient[:3][i]

            # transfer time field to the array of saved time fields

            s.n_time_fields+=1
            s.time_field[s.n_time_fields].arrivaltime=reg.arrivaltime
            tg=reg.time_gradient
            s.time_field[s.n_time_fields].time_gradient=tg

            print('results written to timefield',s.n_time_fields)

            # attach info to the generated time field

            # time field preceding the current one

            s.time_field[s.n_time_fields].prev_tf = prev_tf

            # identify the current time field as the child of the previous time field
            if (prev_tf <= s.n_tf_init):
                s.time_field[prev_tf].next_tf[4+(vtype-1)*4] = s.n_time_fields
            else:
                s.time_field[prev_tf].next_tf[2+(vtype-1)*4] = s.n_time_fields

            # store the index of the time field in the sequence of timefields for this path
            s.init_path[2][vtype].tf_sequence[n] = s.n_time_fields

            # pointers to start and non-start interfaces
            s.time_field[s.n_time_fields].istart = itopc
            s.time_field[s.n_time_fields].inonstart =  ibotc
            s.time_field[s.n_time_fields].reg =  reg

            s.time_field[s.n_time_fields].vtype =  vtype
            prev_tf = s.n_time_fields

            # check for turning rays
            if (np.count_nonzero(itopc.arrivaltime != itopc.starttime) > 0):
                print('turning rays were present on interface',itopc.iface_id)
                s.time_field[s.n_time_fields].turning_rays_present=True
                s.time_field[s.n_time_fields].received_turning_ray = (itopc.arrivaltime != itopc.starttime)

    for m in range(1,n_vtypes):
        for n in range(1,2):
            if (s.init_path[n][m].used):
                if (n==1):
                    print('(a45,i3,a1,11i5)','timefields on upward init path for vtype',m,':', s.init_path[n][m].tf_sequence[:s.init_path[n][m]].n_tf)
                if (n==2):
                    print('(a45,i3,a1,11i5)','timefields on downward init path for vtype',m,':', s.init_path[n][m].tf_sequence[:s.init_path[n][m]].n_tf)
            else:
                if (n==1):
                    print('there was no upward initialization path for vtype',m)
                if (n==2):
                    print('there was no downward initialization path for vtype',m)

def non_alive_neighbors(centernode,reg,grid):
    non_alive_neighbors=False
    i1 = reg.node[centernode].i1
    i2 = reg.node[centernode].i2
    i3 = reg.node[centernode].i3
    # make a list of grid cells of which the new alive node is part
    if (i1 != 0):   # centernode is a regular grid node. Use spatial correlation of grid
        n_concell = 0
        for i in range(0,1):
            if (np.logical_and((i1-i>0),(i1-i<grid.nr))):
                for j in range(0,1):
                    if (np.logical_and((i2-j>0),(i2-j<grid.nlat))):
                        for k in range(0,1):
                            if (np.logical_and((i3-k>0),(i3-k<grid.nlong))):
                                n_concell+=1
                                concell[n_concell].ir=i1-i
                                concell[n_concell].ilat=i2-j
                                concell[n_concell].ilong=i3-k
    else:          # centernode is an intersection node. Use the connections that have been found before
        if (i2 == reg.itop.id):
            isec = reg.itop
        else:
            isec = reg.ibot
        n_concell = 0
        for n in range(1,8):
            icell = isec.ccell_from_inode[n][i3]
            if (icell > 0):
                n_concell+=1
                concell[n_concell].ir= isec.ccells[icell].ir
                concell[n_concell].ilat=isec.ccells[icell].ilat
                concell[n_concell].ilong=isec.ccells[icell].ilong

    for n in range(1,n_concell):       # find the intersection nodes of connected cell n
        if (associated(grid.ccind_from_3dc[concell[n].ir][concell[n].ilat][concell[n].ilong].p)):
            # if so, check if the cell is cut by the top intersection
            # icell is the index of the current connected cell in the list of cells cut by interface reg.itop.
            icell=grid.ccind_from_3dc[concell[n].ir][concell[n].ilat][concell[n].ilong].p[reg.itop.iface_id]
            # if icell == 0 the cell is not cut be the top interface
            if (icell != 0):
                isec = reg.itop
                for jj in range(1,isec.n_inodes[icell]):    #   m is the node number in the regional node list of node  jj in the list of inteface
                    #   nodes that are part of cut cell icell
                    m=isec.rbel_node_id[isec.inodes[jj][icell]]
                    if ( reg.node_status[m] < 0 ):
                        non_alive_neighbours = True
                        #: check the bottom intersection
            icell=grid.ccind_from_3dc[concell[n].ir][concell[n].ilat][concell[n].ilong].p[reg.ibot.iface_id]
            if (icell != 0):
                isec = reg.ibot
                for jj in range(1,isec.n_inodes[icell]):
                    m=isec.rabo_node_id[isec.inodes[jj][icell]]   #                  print *,'its regional node number is ',m,node_is_counted[m],centernode
                    if ( reg.node_status[m] < 0 ):
                        non_alive_neighbours = True
            #: find the regular grid nodes of connected cell n
        for i in range(0,1):
            ii=concell[n].ir+i
            for j in range(0,1):
                jj=concell[n].ilat+j
                for k in range(0,1):
                    kk=concell[n].ilong+k
                    # reduced connectivity for regular nodes if cell is completely regular
                    if (np.logical_or(i1 == 0,np.logical_and(i1 != 0, abs(i1-ii)+abs(i2-jj)+abs(i3-kk) == 1))):
                        if (grid.node_region[ii][jj][kk] == reg.id):
                            # node has to belong to the current region
                            m=grid.rnode_id[ii][jj][kk]
                            if ( reg.node_status[m] < 0 ):
                                non_alive_neighbours = True

def get_source_neighbors(centernode,s,reg,grid):
    i1 = reg.node[centernode].i1
    i2 = reg.node[centernode].i2
    i3 = reg.node[centernode].i3
    # make a list of grid cells of which the new alive node is part
    if (i1 != 0):   # centernode is a regular grid node. Use spatial correlation of grid
        n_concell = 0
        for i in range(0,1):
            if (np.logical_and((i1-i>0),(i1-i<grid.nr))):
                for j in range(0,1):
                    if (np.logical_and((i2-j>0),(i2-j<grid.nlat))):
                        for k in range(0,1):
                            if (np.logical_and((i3-k>0),(i3-k<grid.nlong))):
                                n_concell+=1
                                concell[n_concell].ir=i1-i
                                concell[n_concell].ilat=i2-j
                                concell[n_concell].ilong=i3-k
    else:          # centernode is an intersection node. Use the connections that have been found before
        if (i2 == reg.itop.id):
            isec = reg.itop
        else:
            isec = reg.ibot
        n_concell = 0
        for n in range(1,8):
            icell = isec.ccell_from_inode[n][i3]
            if (icell > 0):
                n_concell+=1
                concell[n_concell].ir= isec.ccells[icell].ir
                concell[n_concell].ilat=isec.ccells[icell].ilat

    for n in range(1,n_concell):      # find the intersection nodes of connected cell n
        if (associated(grid.ccind_from_3dc[concell[n].ir,concell[n].ilat,concell[n].ilong].p)):
            # if so, check if the cell is cut by the top intersection
            # icell is the index of the current connected cell in the list of cells cut by interface reg.itop.
            icell=grid.ccind_from_3dc[concell[n].ir,concell[n].ilat,concell[n].ilong].p[reg.itop.iface_id]
            # if icell == 0 the cell is not cut be the top interface
            if(icell != 0):
                isec = reg.itop
                for jj in range(1,isec.n_inodes[icell]):
                    m=isec.rbel_node_id[isec.inodes[jj][icell]]
                    if ( s.n_cnode == 0 ):
                        s.n_cnode=1
                        mstore[s.n_code] = m
                        s.cnode[s.n_code].i1=reg.node[m].i1
                        s.cnode[s.n_code].i2=reg.node[m].i2
                        s.cnode[s.n_code].i3=reg.node[m].i3
                    else:
                        if (np.count_nonzero(mstore== m) == 0 ):
                            s.n_cnode+=1
                            mstore[s.n_code] = m
                            s.cnode[s.n_code].i1=reg.node[m].i1
                            s.cnode[s.n_code].i2=reg.node[m].i2
                            s.cnode[s.n_code].i3=reg.node[m].i3
            #: check the bottom intersection

            icell=grid.ccind_from_3dc[concell[n].ir][concell[n].ilat][concell[n].ilong].p[reg.ibot.iface_id]
            if (icell != 0):
                isec = reg.ibot
                for jj in range(1,isec.n_inodes[icell]):
                    m=isec.rabo_node_id[isec.inodes[jj][icell]]
                    if ( s.n_cnode == 0 ):
                        s.n_cnode=1
                        mstore[s.n_code] = m
                        s.cnode[s.n_code].i1=reg.node[m].i1
                        s.cnode[s.n_code].i2=reg.node[m].i2
                        s.cnode[s.n_code].i3=reg.node[m].i3
                    else:
                        if ( np.count_nonzero(mstore == m) == 0 ):
                            s.n_cnode+=1
                            mstore[s.n_code] = m
                            s.cnode[s.n_code].i1=reg.node[m].i1
                            s.cnode[s.n_code].i2=reg.node[m].i2
                            s.cnode[s.n_code].i3=reg.node[m].i3
            #: find the regular grid nodes of connected cell n
            for i in range(0,1):
                ii=concell[n].ir+i
                for j in range(0,1):
                    jj=concell[n].ilat+j
                    for k in range(0,1):
                        kk=concell[n].ilong+k
                        if (grid.node_region[ii][jj][kk] == reg.id):
                            # node has to belong to the current region
                            m=grid.rnode_id[ii][jj][kk]
                            if ( s.n_cnode == 0 ):
                                s.n_cnode=1
                                mstore[s.n_code] = m
                                s.cnode[s.n_code].i1=reg.node[m].i1
                                s.cnode[s.n_code].i2=reg.node[m].i2
                                s.cnode[s.n_code].i3=reg.node[m].i3
                            else:
                                if (np.count_nonzero(mstore== m) == 0 ):
                                    s.n_cnode+=1
                                    mstore[s.n_code] = m
                                    s.cnode[s.n_code].i1=reg.node[m].i1
                                    s.cnode[s.n_code].i2=reg.node[m].i2
                                    s.cnode[s.n_code].i3=reg.node[m].i3

def reflect_gradient(isec,tf_prev,vtype):
    if (tf_prev.vtype == vtype):        # simple reflection ( no wave type conversion)
        for n in range(1,isec.nnode):
            isec.time_gradient[:3][n]=isec.time_gradient[:3][n]- 2*np.dot(isec.normal[:3][n],isec.time_gradient[:3][n])*isec.normal[:3][n]
    else:          # reflection with wave type conversion
        if (isec.id == tf_prev.reg.itop.id):   # reflection off the top interface
            vel = isec.vel_bot[:][vtype]
            direction = -1
        else:                                       # reflection off the bottom interface
            vel = isec.vel_top[:][vtype]
            direction = 1
        n_no_ref=0

        for n in range(1,isec.nnode):
            grad_perp=np.dot(isec.normal[:3][n],isec.time_gradient[:3][n])
            grad_par=isec.time_gradient[:3][n]-grad_perp*isec.normal[:3][n]

            det=1/(vel[n]**2) - np.sum(grad_par**2)

            if (det > 0):
                # the refracted ray exists

                grad_perp_reflected=np.sqrt(det)
                isec.time_gradient[:3][n]=grad_par + (np.sign(direction)*(grad_perp_reflected))*isec.normal[:3][n]
            else:
                # no converted reflection possible
                isec.arrivaltime[n]=huge_time
                isec.time_gradient[:3][n]=0
                n_no_ref+=1

        if (n_no_ref > 0):
            print(n_no_ref,' intersection points could not reflect the converted wave')

def refract_gradient(isec,reg,vtype,direction):
    if (isec.id == reg.itop.id):
        vel = isec.vel_top[:][vtype]
    if (isec.id == reg.ibot.id):
        vel = isec.vel_bot[:][vtype]

    n_tot_ref=0
    direction_real = direction

    for n in range(1,isec.nnode):
        grad_perp=np.dot(isec.normal[:3][n],isec.time_gradient[:3][n])
        grad_par=isec.time_gradient[:3][n]-grad_perp*isec.normal[:3][n]

        det=1/(vel[n]**2) - np.sum(grad_par**2)

        if (det > 0):          # the refracted ray exists
            grad_perp_refracted=np.sqrt(det)
            isec.time_gradient[:3][n]=grad_par + (np.sign(direction_real)*(grad_perp_refracted))*isec.normal[:3][n]
        else:
            # total reflection
            isec.arrivaltime[n]=huge_time
            isec.time_gradient[:3][n]=0
            n_tot_ref+=1

    if (n_tot_ref > 0):
        print('total reflection occurred at ',n_tot_ref,' intersection nodes')

def refract_locally(r,lat,long,gridv,tgrad):
    velocity_gradient(r,lat,long,gridv,vgrad[1],vgrad[2],vgrad[3],vel)
#     if (np.sum(vgrad**2) < 10**(-30)):
#         continue  # avoid division by zero if gradient = 0

    normal=vgrad/np.sqrt(np.sum(vgrad**2))
    grad_perp=np.dot(normal,tgrad)
    grad_par=tgrad-grad_perp*normal

    det=1/(vel**2) - np.sum(grad_par**2)

    if (det > 0):
        # the refracted ray exists

        grad_perp_refracted=np.sqrt(det)
        tgrad=grad_par + (np.sign(grad_perp)*(grad_perp_refracted))*normal

def triangulate(t):
    nl=t.npoints
    # nlmax is the maximum number of points
    nlmax=nl
    # ntmax is the maximum number of delaunay triangles built on the points
    ntmax=nlmax*3
    # nhmax is the maximum number of points on the convex hull formed by the lagrangian cloud

    nhmax=nlmax

    # npmax is nlmax

    npmax=nlmax

    # nnpnmax is the maximum number of neighbours per point
    nnpnmax=50

    # nvmax and nmax are the size of working arrays

    nvmax=ntmax
    nmax=3*ntmax+npmax
    dmode=-2
    nmode=0
    clockwise=True
    nohalt_hull=0
    loc=1
    SPfromTR=0
    neighbour=0
    points=0
    field=0
    centres=0
    hulltriangles=0
    nnn=0
    nnlist=0
    ntrilist=0
    vis_tlist=0
    vis_elist=0
    add_tlist=0
    lt_work=False
    ln_work=False

    for i in range(1,nl):
        points[1][i]=t.points[1][i]
        points[2][i]=t.points[2][i]

        # nn_setup (calculates delaunay triangles and other info)

    nn2d_setup(nl,ntmax,nhmax,npmax,nnpnmax,nmax,points,dmode,nmode,clockwise,t.points[1][:],nt,SPfromTR,centres,neighbour,nh,hulltriangles,nohalt_hull,loc,nnn,nnlist,ntrilist,eps,nvmax,vis_tlist,vis_elist,add_tlist,lt_work,ln_work)

    t.ntriangles = nt

    for i in range(1,nt):
        t.points_from_triangle[:3][i]=SPfromTR[:3][i]
        t.triangle_neighbours[:3][i]=neighbour[:3][i]

    # the inverse connectivity
    t.n_triangles_from_point = 0
    for i in range(1,nt):
        for j in range(1,3):
            t.n_triangles_from_point[SPfromTR[j][i]]+=1

    nmax = np.max(t.n_triangles_from_point)
    t.n_triangles_from_point = 0
    for i in range(1,nt):
        for j in range(1,3):
            ipoint = SPfromTR[j][i]
            t.n_triangles_from_point[ipoint] += 1
            t.triangles_from_point[t.n_triangles_from_point[ipoint]][ipoint] = i

def tag_regular_nodes(grid):
    grid.fully_regular = False
    for k in range(2, grid.nlong-1):
        for j in range(2,grid.nlat-1):
            for i in range(2,grid.nr-1):
                not_regular = np.logical_or(associated(grid.ccind_from_3dc[i-1][j-1][k-1].p), associated(grid.ccind_from_3dc[i][j-1][k-1].p), associated(grid.ccind_from_3dc[i-1][j][k-1].p), associated(grid.ccind_from_3dc[i][j][k-1].p), associated(grid.ccind_from_3dc[i-1][j-1][k].p), associated(grid.ccind_from_3dc[i][j-1][k].p), associated(grid.ccind_from_3dc[i-1][j][k].p), associated(grid.ccind_from_3dc[i][j][k].p))
                grid.fully_regular[i][j][k] = ~not_regular

def transfer_refined_region(sreg,reg,tshort):
    nrmin = sgrid.index_r0
    nlatmin = sgrid.index_lat0
    nlongmin = sgrid.index_long0
    for n in range(1,sreg.nnode):
        if (sreg.arrivaltime[n] <= t_short):
            # transfer only times less than the first boundary hit
            i1=sreg.node[n].i1
            i2=sreg.node[n].i2
            i3= sreg.node[n].i3

            if (i1 != 0):
                # the regional node is a regular node
                if (np.logical_and((i1-1)%refinement_factor == 0, (i2-1)%refinement_factor== 0, (i3-1)%refinement_factor == 0)):
                    j1 = nrmin    +   (i1-1)/refinement_factor
                    j2 = nlatmin  +   (i2-1)/refinement_factor
                    j3 = nlongmin +   (i3-1)/refinement_factor

                    reg.arrivaltime[pgrid.rnode_id[j1][j2][j3]] = sreg.arrivaltime[n]
                    reg.time_gradient[:3][pgrid.rnode_id[j1][j2][j3]] = sreg.time_gradient[:3][n]
                    reg.node_status[pgrid.rnode_id[j1][j2][j3]] = 0

            else:
                # the regional node is an intersection node
                if (i2 == sreg.itop.id):
                    isec = reg.itop
                else:
                    isec = reg.ibot

                # coarse grid index coordinates
                xir      = (sintersection[i2].r[i3]-pgrid.r0)/pgrid.dr0
                xilat    = (sintersection[i2].lat[i3]-pgrid.lat0)/pgrid.dlat0
                xilong   = (sintersection[i2].long[i3]-pgrid.long0)/pgrid.dlong0

                # distance from nearest coarse grid coordinate plane

                dxir    = abs(xir - np.floor(xir))
                dxilat  = abs(xilat - np.floor(xilat))
                dxilong = abs(xilong - np.floor(xilong))

                # coordinates of the coarse grid cell containing the refined region node

                j1= min(np.floor(xir)+1,pgrid.nr-1)
                j2= min(np.floor(xilat)+1,pgrid.nlat-1)
                j3= min(np.floor(xilong)+1,pgrid.nlong-1)

                # convert type of inode on refined grid to type of inode on coarse grid

                if (sintersection[i2].intype[i3]!= 0):
                    intype = sintersection[i2].intype[i3]
                else:
                    if (np.logical_and(dxir<interface_tolerance, dxilat<interface_tolerance, dxilong<interface_tolerance)):
                        intype = 0
                    elif (np.logical_and(dxilat < interface_tolerance, dxilong < interface_tolerance)):
                        intype = 1
                    elif (np.logical_and(dxir < interface_tolerance, dxilong < interface_tolerance)):
                        intype = 2
                    elif (np.logical_and(dxir < interface_tolerance, dxilat < interface_tolerance)):
                        intype = 3

                # test whether the refined regional node lies on r,lat or long connection

                if(intype==0):
                    if (np.logical_and(dxir < interface_tolerance, dxilat < interface_tolerance, dxilong < interface_tolerance)):
                        # get the cut cell index of the coarse grid cell
                        if (associated(pgrid.ccind_from_3dc[j1][j2][j3].p)):
                            icell = pgrid.ccind_from_3dc[j1][j2][j3].p[isec.iface_id]
                            # test the nodes of this cut cell for coincidence with the refined inode under consideration
                            for m in range(1,isec.n_inodes[icell]):
                                k=isec.inodes[m][icell]  # the intersection node
                                if (i2 == sreg.itop.id):
                                    kk=isec.rbel_node_id[k]
                                else:
                                    kk=isec.rabo_node_id[k]

                                if (isec.intype[k] == 0):
                                    # only consider nodes of the same type as the refined inode
                                    dist =np.sqrt((sintersection[i2].r[i3]-isec.r[k])**2 + (isec.r[k]*(sintersection[i2].lat[i3]-isec.lat[k]))**2 + (isec.r[k]*isec.coslat[k]*(sintersection[i2].long[i3]-isec.long[k]))**2)
                                    if (dist < pgrid.tolerance):
                                        reg.arrivaltime[kk] = sreg.arrivaltime[n]
                                        reg.time_gradient[:3][kk] = sreg.time_gradient[:3][n]
                                        reg.node_status[kk] = 0
                if(intype==1):
                    # inode lies on an r-connection

                    if (np.logical_and(dxilat < interface_tolerance, dxilong < interface_tolerance)):
                        # get the cut cell index of the coarse grid cell
                        if (associated(pgrid.ccind_from_3dc[j1][j2][j3].p)):
                            icell = pgrid.ccind_from_3dc[j1][j2][j3].p[isec.iface_id]
                            # test the nodes of this cut cell for coincidence with the refined inode under consideration
                            for m in range(1,isec.n_inodes[icell]):
                                k=isec.inodes[m][icell]  # the intersection node
                                if (i2 == sreg.itop.id):
                                    kk=isec.rbel_node_id[k]
                                else:
                                    kk=isec.rabo_node_id[k]

                                if (isec.intype[k] == 1):
                                    # only consider nodes of the same type as the refined inode
                                    dist =np.sqrt((sintersection[i2].r[i3]-isec.r[k])**2 + (isec.r[k]*(sintersection[i2].lat[i3]-isec.lat[k]))**2 + (isec.r[k]*isec.coslat[k]*(sintersection[i2].long[i3]-isec.long[k]))**2)
                                    if (dist < pgrid.tolerance):
                                        reg.arrivaltime[kk] = sreg.arrivaltime[n]
                                        reg.time_gradient[:3][kk] = sreg.time_gradient[:3][n]
                                        reg.node_status[kk] = 0

                if(intype==2):
                    # inode lies on a lat-connection

                    if (np.logical_and(dxir < interface_tolerance, dxilong < interface_tolerance)):
                        if (associated(pgrid.ccind_from_3dc[j1][j2][j3].p)):
                            icell = pgrid.ccind_from_3dc[j1][j2][j3].p[isec.iface_id]
                            for m in range(1,isec.n_inodes[icell]):
                                k=isec.inodes[m][icell]
                                if (i2 == sreg.itop.id):
                                    kk=isec.rbel_node_id[k]
                                else:
                                    kk=isec.rabo_node_id[k]
                                if (isec.intype[k] == 2):
                                    dist =np.sqrt((sintersection[i2].r[i3]-isec.r[k])**2 +(isec.r[k]*(sintersection[i2].lat[i3]-isec.lat[k]))**2 + (isec.r[k]*isec.coslat[k]*(sintersection[i2].long[i3]-isec.long[k]))**2)
                                    if (dist < pgrid.tolerance):
                                        reg.arrivaltime[kk] = sreg.arrivaltime[n]
                                        reg.time_gradient[:3][kk] = sreg.time_gradient[:3][n]
                                        reg.node_status[kk] = 0

                if(intype==3):
                    # inode lies on a long-connection

                        if (np.logical_and(dxilat < interface_tolerance, dxir < interface_tolerance)):
                            if (associated(pgrid.ccind_from_3dc[j1][j2][j3].p)):
                                icell = pgrid.ccind_from_3dc[j1][j2][j3].p[isec.iface_id]
                                for m in range(1,isec.n_inodes[icell]):
                                    k=isec.inodes[m][icell]
                                    if (i2 == sreg.itop.id):
                                        kk=isec.rbel_node_id[k]
                                    else:
                                        kk=isec.rabo_node_id[k]

                                    if (isec.intype[k] == 3):
                                        dist =np.sqrt((sintersection[i2].r[i3]-isec.r[k])**2 + (isec.r[k]*(sintersection[i2].lat[i3]-isec.lat[k]))**2 + (isec.r[k]*isec.coslat[k]*(sintersection[i2].long[i3]-isec.long[k]))**2)
                                        if (dist < pgrid.tolerance):
                                            reg.arrivaltime[kk] = sreg.arrivaltime[n]
                                            reg.time_gradient[:3][kk] = sreg.time_gradient[:3][n]
                                            reg.node_status[kk] = 0

# def write_valid_rays(n,m):
#     if (receiver[n].ray[m].is_multiray):
#         for k in range(1,receiver[n].ray[m].n_subrays):
#             ray = receiver[n].ray[m].subray[k]
#             if (ray.valid):
# #                 write(31,'(5i6)') n,ray.source_id,m,k,ray.nsections # write info

#                 for i in range(1,ray.nsections):
# #                     write(31,'(2i6,2l5)') ray.section[i].npoints,ray.section[i].reg.id, ray.section[i].diffracted,ray.section[i].headwave
#                     for j in range(ray.section[i].npoints,1,-1):
# #                         write(31,'(3f17.8)') ray.section[i].point[:3][j]

#             else:
#                 write(31,'(5i6)') n,ray.source_id,m,k,zero
#     else:
#         k = 0
#         ray = receiver[n].ray[m]
#         if (ray.valid):
#             write(31,'(5i6)') n,ray.source_id,m,zero,ray.nsections
#             for i in range(1,ray.nsections):
#                 write(31,'(2i6,2l5)') ray.section[i].npoints,ray.section[i].reg.id, ray.section[i].diffracted,ray.section[i].headwave
#                 for j in range(ray.section[i].npoints,1,-1):
#                     write(31,'(3f17.8)') ray.section[i].point[:3][j]
#         else:
#             write(31,'(5i6)') n,ray.source_id,m,zero,zero

# def write_frechet_derivatives(n,m):
#     if (receiver[n].ray[m].is_multiray):
#         for k in range(1,receiver[n].ray[m].n_subrays):
#             ray = receiver[n].ray[m].subray[k]
#             if (ray.valid):
#                 write(21,'(5i6)') n,ray.source_id,m,k,ray.n_pdev # write info
#                 for i in range(1,ray.n_pdev):
#                     write(21,'(i10,f17.8)') ray.pdev_indx[i],ray.pdev[i]
#             else:
#                 write(21,'(5i6)') n,ray.source_id,m,k,zero
#     else:
#         k = 0
#         ray = receiver[n].ray[m]
#         if (ray.valid):
#             write(21,'(5i6)') n,ray.source_id,m,k,ray.n_pdev
#             for i in range(1,ray.n_pdev):
#                 write(21,'(i10,e17.8)') ray.pdev_indx[i],ray.pdev[i)]
#         else:
#             write(21,'(5i6)') n,ray.source_id,m,k,zero

# def load_source_timefields(s):
#     nfile=s.nfile
#     open(nfile,form='unformatted')
#     for n in range(1,s.n_time_fields):
#         read(nfile) s.time_field[n].arrivaltime
#         read(nfile) s.time_field[n].time_gradient
#     close(nfile)

def write_arrival_time_grid(src,path):
    pgrid.arrivaltime=huge_time
    for n in range(path.first_tf_to_save, path.n_tf):
        tf= src.time_field[path.tf_sequence[n]]
        reg = tf.reg
        for i in range(1, reg.nnode):
            i1=reg.node[i].i1
            i2=reg.node[i].i2
            i3=reg.node[i].i3

            if (i1 != 0):
                pgrid.arrivaltime[i1][i2][i3]=min(tf.arrivaltime[i],pgrid.arrivaltime[i1][i2][i3])
            if (i1==0):
                r=reg.r[i]-pgrid.r0 ; lat=reg.lat[i]-pgrid.lat0 ; long=reg.long[i]-pgrid.long0

                ir=np.floor(r/pgrid.dr0)+1
                ilat=np.floor(lat/pgrid.dlat0)+1
                ilong=np.floor(long/pgrid.dlong0)+1

                dxr=abs(r-pgrid.dr0*(ir-1))
                dxlat=reg.r[i]*abs(lat-pgrid.dlat0*(ilat-1))
                dxlong=reg.r[i]*cos(reg.lat[i])*abs(long-pgrid.dlong0*(ilong-1))

                ntype=intersection[i2].intype[i3]

                if(ntype==0):
                    pgrid.arrivaltime[ir][ilat][ilong]= min(tf.arrivaltime[i],pgrid.arrivaltime[ir][ilat][ilong])
                if(ntype==1):
                    if (dxr<=pgrid.tolerance):
                        pgrid.arrivaltime[ir][ila][ilong]= min(tf.arrivaltime[i],pgrid.arrivaltime[ir][ilat][ilong])
                if(ntype==2):
                    if (dxlat<=pgrid.tolerance):
                        pgrid.arrivaltime[ir,ilat][ilong]= min(tf.arrivaltime[i],pgrid.arrivaltime[ir][ilat][ilong])
                if(ntype==3):
                    if (dxlong<=pgrid.tolerance):
                        pgrid.arrivaltime[ir][ilat][ilong]= min(tf.arrivaltime[i],pgrid.arrivaltime[ir][ilat][ilong])
                if(pgrid.arrivaltime>10**10):
                    pgrid.arrivaltime=-1.0
#                     write(19,*) src.id,path.id,path.first_tf_to_save
#                 for i3 in range(1,pgrid.nlong):
#                     for i2 in range(1,pgrid.nlat):
#                         for i1 in range(1,pgrid.nr):
# #                             write(19,'(f12.5)') pgrid.arrivaltime[i1][i2][i3]

In [5]:
def propagate(regin,vtype):
    reg = regin
    grid = reg.grid
    velocity = reg.velocity[:][vtype]
    velgrid = vgrid[reg.ivgrid][vtype]

    n_det_negative = 0

    if (reg.id == 99):
        diat=True
        testnode=pgrid.rnode_id(37,9,5)
#     write (24,*) 'testnode set to',testnode
#     write (24,'(a20,3f12.5)') 'test coordinates',reg.r(testnode),reg.lat(testnode),reg.long(testnode)
#     write (24,'(3i8)') reg.node(testnode).i1,reg.node(testnode).i2,reg.node(testnode).i3
    else:
        diat=False
        # the time gradient derived during regular updates when fewer than 3 nodes are used can be very inaccurate
        # set this flag in that case and correct afterwards


    suspect_time_gradient = False

    # precalculate required sines and cosines

    cosla=np.cos(reg.lat)
    sinla=np.sin(reg.lat)
    coslo=np.cos(reg.long)
    sinlo=np.sin(reg.long)


    # set up the initial tree

    node_is_counted=0
    ntree = 0

    for n in range(1,reg.nnode):
        if (reg.node_status[n] == 1):
            add_node_to_tree(n)
            #start main propagation loop

    first_step=True
    while (ntree > 0):
        # take the node with the smallest time from the narrowband, set it to alive and adjust the tree
        newalive=node_from_tree_ind[1]
        reg.node_status[newalive] = 0
        remove_root_from_tree()

        if (np.logical(ntree == 0, ~ first_step)):
            exit  # stop when the tree structure is empty, but not in the first step since there may
                                               # be only a single starting point (the source)
        first_step=False
        if (diag):
            print('root removed',newalive)
#         if (diag):
#             print('(i5,3f12.5,3i5)',newalive,reg.r(newalive),reg.lat(newalive),reg.long(newalive),reg.node(newalive)
        # find the neighbours of newalive, a list of node numbers is returned in the array connode
        find_connected_nodes(newalive)
        if (diag):
            print('neighbour list found',n_connode)
        if (diag):
            for n in range(1,n_connode):
                print('(2i5,3f12.5,3i5)',n,connode[n],reg.r[connode[n]],reg.lat[connode[n]],reg.long[connode[n]],reg.node[connode[n]])
            print('arrivaltime at newalive is',reg.arrivaltime[newalive])

            # save the list of neighbours, because the array connode will be used again when finding neighbours of the
            # neighbours of newalive that have to be updated

        n_trial=n_connode
        trialnode[1:n_trial]=connode[1:n_trial]

        #  loop over neighbours and take appropopriate action

        for nt in range(1,n_trial):
            # if the neighbour is already in the narrow band, update it
            if (reg.node_status[trialnode[nt]] > 0):
                oldtime=reg.arrivaltime[trialnode[nt]]
                update_time(trialnode[nt])
            if (diag):
                print('(i5,a6,f12.5,a6,f12.5)',nt,'nb ',reg.arrivaltime[trialnode[nt]],'was',oldtime)
            if (reg.arrivaltime[trialnode[nt]] < oldtime):
                update_tree(trialnode[nt])

                # if the neighbour is far, add it to the narrow band

            if (reg.node_status[trialnode[nt]] == -1):
                reg.arrivaltime[trialnode[nt]] = huge_time
                oldtime=reg.arrivaltime[trialnode[nt]]
                update_time(trialnode[nt])

            if (diag):
                print('(i5,a6,f12.5)',nt,'far',reg.arrivaltime[trialnode[nt]])
            if (reg.arrivaltime[trialnode[nt]] < oldtime):
                add_node_to_tree(trialnode[nt])
                # loop over neighbours of newalive

    for n in range(1,reg.nnode):
        if (suspect_time_gradient[n]):
            fit_gradient_at_node(reg,n,reg.time_gradient[1][n],reg.time_gradient[2][n],reg.time_gradient[3][n])

def find_connected_nodes(centernode):
    i1 = reg.node[centernode].i1
    i2 = reg.node[centernode].i2
    i3 = reg.node[centernode].i3
    if (i1 != 0):
        if (grid.fully_regular[i1][i2][i3]):
            n_connode = 6
            connode[1]=grid.rnode_id[i1-1][i2][i3]
            connode[2]=grid.rnode_id[i1+1][i2][i3]
            connode[3]=grid.rnode_id[i1][i2-1][i3]
            connode[4]=grid.rnode_id[i1][i2+1][i3]
            connode[5]=grid.rnode_id[i1][i2][i3-1]
            connode[6]=grid.rnode_id[i1][i2][i3+1]

            # make a list of grid cells of which the new alive node is part

    if (i1 != 0):    # centernode is a regular grid node. Use spatial correlation of grid
        n_concell = 0
        for i in range(0,1):
            if (np.logical_and((i1-i>0), (i1-i<grid.nr))):
                for j in range(0,1):
                    if (np.logical_and((i2-j>0),(i2-j<grid.nlat))):
                        for k in range(0,1):
                            if (np.logical_and((i3-k>0),(i3-k<grid.nlong))):
                                n_concell +=1
                                concell[n_concell].ir=i1-i
                                concell[n_concell].ilat=i2-j
                                concell[n_concell].ilong=i3-k
    else:          # centernode is an intersection node. Use the connections that have been found before
        if (i2 == reg.itop.id):
            isec = reg.itop
        else:
            isec = reg.ibot

        n_concell = 0
        for n in range(1,8):
            icell = isec.ccell_from_inode[n][i3]
            if (icell > 0):
                n_concell += 1
                concell[n_concell].ir= isec.ccells[icell].ir
                concell[n_concell].ilat=isec.ccells[icell].ilat
                concell[n_concell].ilong=isec.ccells[icell].ilong

                # make a list of nodes in the connected cells
                # make sure the centernode itself is not counted as one of the neighbours

    node_is_counted[centernode]=centernode

    n_connode=0

    for n in range(1,n_concell):
        # if centernode is a regular node (i1 /= 0) , the cell is potentially regular
        concell_is_regular[n] = (i1 != 0)
        # find the intersection nodes of connected cell n

        if (associated(grid.ccind_from_3dc[concell[n].ir][concell[n].ilat][concell[n].ilong].p)):
            # if so, check if the cell is cut by the top intersection
            # icell is the index of the current connected cell in the list of cells cut by interface reg.itop.

            icell = grid.ccind_from_3dc[concell[n].ir][concell[n].ilat][concell[n].ilong].p[reg.itop.iface_id]

            # if icell == 0 the cell is not cut be the top interface

            if(icell != 0):
                concell_is_regular[n] = False
                isec = reg.itop

                for jj in range(1,isec.n_inodes[icell]):
                    #   m is the node number in the regional node list of node  jj in the list of inteface nodes that are part of cut cell icell
                    m=isec.rbel_node_id[isec.inodes[jj][icell]]
                    if ( node_is_counted[m]!= centernode):   # if node is not yet counted as a neighbour of centernode
                        n_connode += 1
                        connode[n_connode]=m
                        node_is_counted[m] = centernode

                        # then check the bottom intersection

            icell = grid.ccind_from_3dc[concell[n].ir][concell[n].ilat][concell[n].ilong].p[reg.ibot.iface_id]
            if (icell != 0):
                concell_is_regular[n] = False
                isec = reg.ibot

                for jj in range(1,isec.n_inodes[icell]):
                    m=isec.rabo_node_id[isec.inodes[jj][icell]]
                    if (node_is_counted[m] != centernode):
                        n_connode += 1
                        connode[n_connode]=m
                        node_is_counted[m] = centernode

                        # then find the regular grid nodes of connected cell n
        for i in range(0,1):
            ii=concell[n].ir+i
            for j in range(0,1):
                jj=concell[n].ilat+j
                for k in range(0,1):
                    kk=concell[n].ilong+k

                    # reduced connectivity for regular nodes if cell is completely regular
                    if (np.logical_or(np.logical_and(i1 != 0, abs(i1-ii)+abs(i2-jj)+abs(i3-kk) == 1)),(~concell_is_regular[n])):
                        if (grid.node_region[ii][jj][kk] == reg.id):
                            # node has to belong to the current region
                            m=grid.rnode_id[ii][jj][kk]
                            if (node_is_counted[m] != centernode):
                                n_connode +=1
                                connode[n_connode]=m
                                node_is_counted[m] = centernode

def find_connected_cells(centernode):
    i1 = reg.node[centernode].i1
    i2 = reg.node[centernode].i2
    i3 = reg.node[centernode].i3
    # make a list of grid cells of which the new node is part
    if (i1 != 0):   # centernode is a regular grid node. Use spatial correlation of grid
        n_concell = 0
        for i in range(0,1):
            if (np.logical_and((i1-i>0),(i1-i<grid.nr))):
                for j in range(0,1):
                    if (np.logical_and((i2-j>0),(i2-j<grid.nlat))):
                        for k in range(0,1):
                            if (np.logical_and((i3-k>0),(i3-k<grid.nlong))):
                                n_concell += 1
                                concell[n_concell].ir=i1-i
                                concell[n_concell].ilat=i2-j
                                concell[n_concell].ilong=i3-k
    else:          # centernode is an intersection node. Use the connections that have been found before
        if (i2 == reg.itop.id):
            isec = reg.itop
        else:
            isec = reg.ibot
        n_concell = 0
        for n in range(1,8):
            icell = isec.ccell_from_inode[n][i3]
            if (icell > 0):
                n_concell += 1
                concell[n_concell].ir= isec.ccells[icell].ir
                concell[n_concell].ilat=isec.ccells[icell].ilat
                concell[n_concell].ilong=isec.ccells[icell].ilong

                # make a list of nodes in the connected cells

    n_connode=0
    for n in range(1,n_concell):
        # if centernode is a regular node (i1 != 0) , the cell is potentially regular
        concell_is_regular[n] = (i1 != 0)
        # the intersection nodes of these cells

        # test whether the cell is cut by any interface, in that case the pointer to the local list of
        # interfaces cutting the cell has been allocated

        if (associated(grid.ccind_from_3dc[concell[n].ir][concell[n].ilat][concell[n].ilong].p)):
            # if so, check the top intersection
            # icell is the index of the current connected cell in the list of cells cut by interface reg.itop
            # if the current connected cell is not cut by this interface icell == 0
            icell = grid.ccind_from_3dc[concell[n].ir][concell[n].ilat][concell[n].ilong].p[reg.itop.iface_id]
            if(icell != 0):
                concell_is_regular[n]= False
                isec = reg.itop
                for jj in range(1,isec.n_inodes[icell]):
                    m=isec.rbel_node_id[isec.inodes[jj][icell]]
                    # take only alive nodes (note that this excludes centernode itself)
                    if(reg.node_status[m] == 0 ):
                        n_connode += 1
                        connode[n_connode]=m

            #: for the same for the bottom intersection

            icell = grid.ccind_from_3dc[concell[n].ir][concell[n].ilat][concell[n].ilong].p[reg.ibot.iface_id]
            if (icell != 0):
                concell_is_regular[n]=False
                isec = reg.ibot
                if (icell < 1):
                    print('icell < 1',icell)
                if (icell > isec.n_ccells):
                    print('icell > n_ccells',icell,isec.n_ccells)
                    print('cell',concell[n].ir,concell[n].ilat,concell[n].ilong)
                    print('interface',ii)
                    print(grid.ccind_from_3dc[concell[n].ir][concell[n].ilat][concell[n].ilong].p[1:4])
                for jj in range(1,isec.n_inodes[icell]):
                    m=isec.rabo_node_id[isec.inodes[jj][icell]]
                    if (reg.node_status[m] == 0 ):
                        n_connode +=1
                        connode[n_connode]=m
                        # the regular grid nodes of these cells
        for i in range(0,1):
            ii=concell[n].ir+i
            for j in range(0,1):
                jj=concell[n].ilat+j
                for k in range(0,1):
                    kk=concell[n].ilong+k

                    # reduced connectivity for regular nodes if cell is completely regular
                    if (np.logical_or((np.logical_and((i1 != 0),(abs(i1-ii)+abs(i2-jj)+abs(i3-kk) == 1))),(~concell_is_regular[n]))):
                        if (grid.node_region[ii][jj][kk] == reg.id):  # node has to belong to the current region
                            m=grid.rnode_id[ii][jj][kk]
                            if (reg.node_status[m]== 0):  # node has to be alive(note that this excludes centernode itself)
                                n_connode +=1
                                connode[n_connode]=m

    concell_nnode[n]=n_connode

    # store the starting index and length of the list of alive nodes for each cell, since updating is done by cell

    concell_nbase[1]=1
    for n in range(1,n_concell):
        concell_nnode[n]=concell_nnode[n]-concell_nbase[n]+1
        if(n < n_concell):
            concell_nbase[n+1]=concell_nbase[n]+concell_nnode[n]

def update_time(update_node):
    i1 = reg.node[update_node].i1
    i2 = reg.node[update_node].i2
    i3 = reg.node[update_node].i3
    if (i1 != 0):
        if (grid.fully_regular[i1][i2][i3]):
            #         print *,'node is fully regular'     # deldiag
            for k in range(-1,1,2):
                for j in range(-1,1,2):
                    for i in range(-1,1,2):
                        nanode=0
                        id = grid.rnode_id[i1+i][i2][i3]
                        if (reg.node_status[id] == 0):
                            nanode += 1
                            anode[nanode]= id
                        id = grid.rnode_id[i1][i2+j][i3]
                        if (reg.node_status[id] == 0):
                            nanode += 1
                            anode[nanode]= id
                        id = grid.rnode_id[i1][i2][i3+k]
                        if (reg.node_status[id] == 0):
                            nanode +=1
                            anode[nanode]= id
                        if (nanode > 0) :
                            # put newalive at position 1 if it is there
                            for n in range(1,nanode):
                                if (anode[n] == newalive):
                                    anode[n]=anode[1]
                                    anode[1]=newalive
                            if (anode[1] == newalive):
                                regular_update2(update_node,anode[1:nanode],nanode)

                                # first find the cells to which the node to be updated is connected, and the list of
                                # ALIVE nodes in these cells
    find_connected_cells(update_node)
    # now loop over the connected cells, and if possible use the alive nodes
    # in each cell to find a new arrival time at the node to be updated
    for n in range(1,n_concell):
        if (concell_nnode[n] > 0):
            # if there are any alive nodes in this cell
            # copy alive nodes in this connected cell to local array
            nanode=concell_nnode[n]
            if (nanode > 30):
                print('nanode =',nanode)
                for i in range(1,nanode):
                    print('(4i5,4f12.5)',i,reg.node[anode[i]].i1,reg.node[anode[i]].i2,reg.node[anode[i]].i3, reg.r[anode[i]],reg.lat[anode[i]],reg.long[anode[i]])
            anode[1:nanode]=connode[concell_nbase[n]:concell_nbase[n]+nanode-1]
            # put newalive at position 1 if it is there
            for i in range(1,nanode):
                if (anode[i] == newalive):
                    j=anode[1]
                    anode[1]=anode[i]
                    anode[i]=j
                    # test if newalive is among the nodes that will be used for the update
                    # otherwise the update has already been forne and fores not have to be repeated
                    if (anode[1] == newalive):
                        if (concell_is_regular[n]):
                            # for a regular fast marching update, second order if possible, in this octant
                            regular_update2(update_node,anode[1:nanode],nanode)
                        else:
                            # for an irregular update in this octant
                            if (nanode==1): # there is 1 alive node in the cell
                                time_from_1_node(update_node,anode[1])
                            if (nanode==2):     # there are 2 alive nodes in the cell
                                time_from_1_node(update_node,anode[1])
                                time_from_2_nodes(update_node,anode[1],anode[2])
                            if (nanode==3):     # there are 3 alive nodes in the cell
                                time_from_1_node(update_node,anode[1])
                                time_from_2_nodes(update_node,anode[1],anode[2])
                                time_from_2_nodes(update_node,anode[1],anode[3])
                                time_from_3_nodes(update_node,anode[1],anode[2],anode[3])
                            if (nanode==4):     # there are more than 3 alive nodes in the cell
                                time_from_1_node(update_node,anode[1])
                                for jl in range(2,nanode):
                                    time_from_2_nodes(update_node,anode[1],anode[jl])
                                for jl in range(2,nanode-1):
                                    for kl in range(jl+1,nanode):
                                        time_from_3_nodes(update_node,anode[1],anode[jl],anode[kl])

def time_from_1_node(unode,node1):
    xu[1]=reg.r[unode]*cosla[unode]*coslo[unode]
    xu[2]=reg.r[unode]*cosla[unode]*sinlo[unode]
    xu[3]=reg.r[unode]*sinla[unode]

    x1[1]=reg.r[node1]*cosla[node1]*coslo[node1]
    x1[2]=reg.r[node1]*cosla[node1]*sinlo[node1]
    x1[3]=reg.r[node1]*sinla[node1]

    dist=np.sqrt(np.sum((xu-x1)**2))

    if (dist < 0.01*pgrid.tolerance):
        reg.arrivaltime[unode] = reg.arrivaltime[node1]
        reg.time_gradient[1:3][unode] =  reg.time_gradient[1:3][node1]

    ttime= dist/(0.5*(velocity[unode]+velocity[node1]))
    atime= reg.arrivaltime[node1] + ttime

    if (atime < reg.arrivaltime[unode]):
        reg.arrivaltime[unode] = atime

        # convert time gradient back to spherical

        b=(xu-x1)/(dist*velocity[unode])
        a[1][1]=cosla[unode]*coslo[unode]
        a[2][1]=-sinla[unode]*coslo[unode]
        a[3][1]=-sinlo[unode]
        a[1][2]=cosla[unode]*sinlo[unode]
        a[2][2]=-sinla[unode]*sinlo[unode]
        a[3][2]= coslo[unode]
        a[1][3]=sinla[unode]
        a[2][3]= cosla[unode]
        a[3][3]= 0

        reg.time_gradient[1:3][unode] = np.matmul(a,b)
        refract_locally(reg.r[unode],reg.lat[unode],reg.long[unode],velgrid,reg.time_gradient[1:3][unode])

def time_from_2_nodes(unode,anode1,anode2):
    node1=anode1
    node2=anode2
    # order from latest to earliest arrival time
    if (reg.arrivaltime[node1] < reg.arrivaltime[node2]):
        i=node1
        node1=node2
        node2=i
    # convert nodes to Cartesian

    xu[1]=reg.r[unode]*cosla[unode]*coslo[unode]
    xu[2]=reg.r[unode]*cosla[unode]*sinlo[unode]
    xu[3]=reg.r[unode]*sinla[unode]

    x1[1]=reg.r[node1]*cosla[node1]*coslo[node1]
    x1[2]=reg.r[node1]*cosla[node1]*sinlo[node1]
    x1[3]=reg.r[node1]*sinla[node1]

    x2[1]=reg.r[node2]*cosla[node2]*coslo[node2]
    x2[2]=reg.r[node2]*cosla[node2]*sinlo[node2]
    x2[3]=reg.r[node2]*sinla[node2]

    # first define some local variables (r1,r2 difference vectors between node1,node2 and unode)

    r1=x1-xu
    r2=x2-xu

    # first check if nodes coincide, if so copy times and gradients and return

    if (np.sqrt(np.sum(r1**2)) < 0.01*pgrid.tolerance):
        reg.arrivaltime[unode] = reg.arrivaltime[node1]
        reg.time_gradient[1:3][unode] =  reg.time_gradient[1:3][node1]

    if (np.sqrt(np.sum(r2**2)) < 0.01*pgrid.tolerance):
        reg.arrivaltime[unode] = reg.arrivaltime[node2]
        reg.time_gradient[1:3][unode] =  reg.time_gradient[1:3][node2]

    if (np.sqrt(np.sum((x2-x1)**2)) < 0.01*pgrid.tolerance):
        time_from_1_node(unode,node1)

        # construct the two linear equations

        b[1]=reg.arrivaltime[node1]-reg.arrivaltime[node2]
        b[2]=0

        a[1][1:3]=(x1-x2)/(0.5*(velocity[node1]+velocity[node2]))

        a[2][1]=r1[2]*r2[3]-r1[3]*r2[2]
        a[2][2]=r1[3]*r2[1]-r1[1]*r2[3]
        a[2][3]=r1[1]*r2[2]-r1[2]*r2[1]

        n0=a[2][:]    # save the normal to the plane containing the 3 points (2 alive+update) for future use
        # the direction of the line that is the intersection between the two planes defined by the
        # two linear equations is the cross product  q = a(1,1:3) x a(2,1:3)

        q[1]=a[1][2]*a[2][3]-a[1][3]*a[2][2]
        q[2]=a[1][3]*a[2][1]-a[1][1]*a[2][3]
        q[3]=a[1][1]*a[2][2]-a[1][2]*a[2][1]
        # a third equation is added that defines the plane that is normal to the direction of the
        # intersection line and goes through (0,0,0)

        b[3]=0
        a[3][1]=q[1]
        a[3][2]=q[2]
        a[3][3]=q[3]
        # now find the intersection point of the intersection line with the plane that is normal to it
        # and goes through (0,0,0). This gives the minimum length support vector

        ludcmp(a,indx,status)
        if (status == 1):
            print(a[1][1:3],b[1])
            print(a[2][1:3],b[2])
            print(a[3][1:3],b[3])

        lubksb(a,indx,b)
        p=b
        # we now have the intersection line defined as p+uq , where u is a scalar variable
        # solve the quadratic normalization constraint |n| = 1 for u
        ae=q[1]*q[1]+q[2]*q[2]+q[3]*q[3]
        be=2*(p[1]*q[1]+p[2]*q[2]+p[3]*q[3])
        ce=p[1]*p[1]+p[2]*p[2]+p[3]*p[3]-1
        det=be*be-4*ae*ce

        u1=0.5*(-be+np.sqrt(det))/ae
        u2=0.5*(-be-np.sqrt(det))/ae
        # the two solutions for the wave vector

        wn1=p+u1*q
        wn2=p+u2*q

        # now test if either of these two solutions is valid (obeys the causality constraint)
        # -wn  must lie within the wedge defined by the planes containing x1-x0 or x2-x0
        # and the normal to the plane of the three points x0,x1,x2, which is just the vector a(2,:)
        # calculated above (saved in n0 before calling ludcmp because a is modified there)
        # calculate the inward pointing normals of the two planes defining the wedge

        n1[1]=n0[2]*r1[3]-n0[3]*r1[2]
        n1[2]=n0[3]*r1[1]-n0[1]*r1[3]
        n1[3]=n0[1]*r1[2]-n0[2]*r1[1]
        n1=n1/np.sqrt(np.sum(n1**2))
        if (n1[1]*r2[1]+n1[2]*r2[2]+n1[3]*r2[3] < 0):
            n1=-n1
            # define the sign of the normal so the other alive point is
            # on the positive side

        n2[1]=n0[2]*r2[3]-n0[3]*r2[2]
        n2[2]=n0[3]*r2[1]-n0[1]*r2[3]
        n2[3]=n0[1]*r2[2]-n0[2]*r2[1]
        n2=n2/np.sqrt(sum(n2**2))
        if (n2[1]*r1[1]+n2[2]*r1[2]+n2[3]*r1[3] < 0):
            n2=-n2
            # define the sign of the normal so the other alive point is
            # on the positive side
        vscount=0
        if (np.logical_and(np.dot(wn1,n1) <= 0, np.dot(wn1,n2) <= 0)):
            atime= reg.arrivaltime[node1]-np.dot(wn1,r1)/(0.5*(velocity[unode]+velocity[node1]))
            if (atime >= reg.arrivaltime[node1]):
                if (atime < reg.arrivaltime[unode]):
                    reg.arrivaltime[unode] = atime

                    b=wn1/(np.sqrt(np.sum(wn1**2))*velocity[node1])
                    a[1][1]=cosla[unode]*coslo[unode]
                    a[2][1]=-sinla[unode]*coslo[unode]
                    a[3][1]=-sinlo[unode]

                    a[1][2]=cosla[unode]*sinlo[unode]
                    a[2][2]=-sinla[unode]*sinlo[unode]
                    a[3][2]= coslo[unode]

                    a[1][3]=sinla[unode]
                    a[2][3]= cosla[unode]
                    a[3][3]= 0

                    reg.time_gradient[1:3][unode] = np.matmul(a,b)
                    x1=reg.time_gradient[1:3][unode]

                    refract_locally(reg.r[unode],reg.lat[unode],reg.long[unode],velgrid, reg.time_gradient[1:3][unode])

                    vscount +=1
                    if (np.logical_and(dot_product(wn2,n1) <= 0, np.dot(wn2,n2) <= 0)):
                        atime= reg.arrivaltime[node1]-np.dot(wn2,r1)/(0.5*(velocity[unode]+velocity[node1]))
                        if (atime >= reg.arrivaltime[node1]):
                            if (atime < reg.arrivaltime[unode]):
                                reg.arrivaltime[unode] = atime
                                b=wn2/(np.sqrt(np.sum(wn2**2))*velocity[node1])
                                a[1][1]=cosla[unode]*coslo[unode]
                                a[2][1]=-sinla[unode]*coslo[unode]
                                a[3][1]=-sinlo[unode]

                                a[1][2]=cosla[unode]*sinlo[unode]
                                a[2][2]=-sinla[unode]*sinlo[unode]
                                a[3][2]= coslo[unode]

                                a[1][3]=sinla[unode]
                                a[2][3]= cosla[unode]
                                a[3][3]= 0

                                reg.time_gradient[1:3][unode] = np.matmul(a,b)

                                refract_locally(reg.r[unode],reg.lat[unode],reg.long[unode],velgrid, reg.time_gradient[1:3][unode])

                    vscount +=1
                    
def associated(x):
    if (x!=0):
        return True
    else:
        return False

Main Code

In [44]:
# Initialise objects:

initialise_propagation_grid(pgrid)
initialise_velocity_grids(vgrid)
initialise_interfaces(interface,n_interfaces)

# Initialise intersections:

for i in range(0,n_intersections):
#     intersection_defaults(intersection[i])
    intersection[i].id_=i

for i in range(0,n_intersections):
    find_intersection(intersection[i],interface[i],pgrid)

print("Main grid intersections initialized.")

n_regions=n_interfaces - 1
region(n_regions)
for i in range(0,n_regions):
    region_defaults(region[i])
    region[i].id_=i

    # Pointer implementation:
    region[i].i_top= intersection[i]
    region[i].i_bot= intersection[i+1]

    if (region[i].i_top.n_node==0):
        print("Region",i,"doesn't exist")
        region[i].nnode=0

    define_region(region[i],region[i].i_top,region[i].i_bot,pgrid)

print("Main grid regions initialized.")

tag_regular_nodes(pgrid)

velocities_to_grid(pgrid)

for i in range(0,n_intersections):
    velocities_to_intersection(intersection[i])

for i in range(0,n_regions):
    velocities_to_region(region[i],pgrid)

print("Velocities on main grid, its intersections and regions evaluated")

## (Define s - class)

for i in range(0,n_sources):
    if s[i].is_teleseismic==True:
        n_teleseismic+=1
        s[i].teleseismic_phase=0
        s[i].r=earth_radius - s[i].r
        s[i].lat=np.radians(s[i].lat)
        s[i].long=np.radians(s[i].long)
        s[i].coslat=np.cos(s[i].lat)
    else:
        s[i].r=earth_radius - s[i].r
        s[i].lat=np.radians(s[i].lat)
        s[i].long=np.radians(s[i].long)
        s[i].coslat=np.cos(s[i].lat)

    initialize_source(s[i],pgrid)

    n_paths=20
    s[i].path(n_paths)
    for j in range(0,n_paths):
        path_defaults(s[i].path)

    for j in range(0,n_paths):
        s[i].path.id=j
        s[i].path.n_tf=(read_value) #read value code needed
        s[i].path.sequence[2*n_tf] #read data
        s[i].path.tf_sequence[2*n_tf]
        s[i].path.vtype_sequence[2*n_tf] #read data

    for step in range(0,n_tf):
        if (s[i].path.sequence[2*step-1] == s[i].path.sequence[2*step]):
            s[i].path.refstep=step
            s[i].path.fitting_interface=s[i].path.sequence[2*step]

## Test consistency with source position (lines 303-533)

npp_receivers=np.zeros(len(n_receivers))
ipp_receivers=np.ones(len(n_receivers))

for i in range(0,n_receivers):
    for j in range(0,r[i].n_rays):
        path_id=r[i].ray[j].raypath_id
        r[i].ray[j].source.path[path_id].used=1 #check source and s[]

        if (r[i].ray[j].source.path[path_id].refstep!=0):
            npp_receivers[i]+=1

#  Add receivers of pp-type phases to the source list, and construct their path sequences by inverting the tail end of the original pp-type sequence

for i in range(0,n_receivers):
    if (npp_receivers[i]>0):
        r[i].path_equivalent=np.zeros(r[i].n_rays)
    for j in range(0,r[i].n_rays):
        path_id=r[i].ray[j].raypath_id

        if (r[i].ray[j].source.path[path_id].refstep!=0):
            if (r[i].source_equivalent==0):
                n_sources_ppinc+=1
                r[i].source_equivalent=n_sources_ppinc
                s[r[i].source_equivalent].r=r[i].r
                s[r[i].source_equivalent].lat=r[i].lat
                s[r[i].source_equivalent].long=r[i].long
                s[r[i].source_equivalent].coslat=r[i].coslat

                s[r[i].source_equivalent].is_local=True

                initialize_source(s,pgrid) # use pyproj,shapely etc

                r[i].path_equivalent[m]=1

                s[r[i].source_equivalent].n_paths=npp_receivers[i]
                s[r[i].source_equivalent].path=np.zeros(s[r[i].source_equivalent].n_paths)

                for k in range(0,s[r[i].source_equivalent].n_paths):
                    path_defaults(s[r[i].source_equivalent].path[k])

            else:
                r[i].path_equivalent[j]=ipp_receivers[i]

        recpath_id=ipp_receivers[i]

        s[r[i].source_equivalent].path[recpath_id].n_tf=r[i].ray[j].source.path[path_id].n_tf-r[i].ray[j].source.path[path_id].refstep
        s[r[i].source_equivalent].path[recpath_id].id=ipp_receivers[i]
        s[r[i].source_equivalent].path[recpath_id].valid=1
        s[r[i].source_equivalent].path[recpath_id].used=1
        s[r[i].source_equivalent].path[recpath_id].refstep=s[r[i].source_equivalent].path[recpath_id].n_tf + 1
        s[r[i].source_equivalent].path[recpath_id].fitting_interface=r[i].ray[j].source.path[path_id].fitting_interface

        s[r[i].source_equivalent].path[recpath_id].sequence=np.zeros(2*s[r[i].source_equivalent].path[recpath_id].n_tf)
        s[r[i].source_equivalent].path[recpath_id].tf_sequence=np.zeros(s[r[i].source_equivalent].path[recpath_id].n_tf)
        s[r[i].source_equivalent].path[recpath_id].vtype_sequence=np.zeros(s[r[i].source_equivalent].path[recpath_id].n_tf)

        for m in range(0,2*s[r[i].source_equivalent].path[recpath_id].n_tf,2):
            s[r[i].source_equivalent].path[recpath_id].sequence[m]=r[i].ray[j].source.path[path_id].sequence[2*r[i].ray[j].source.path[path_id].n_tf-m+1]
            s[r[i].source_equivalent].path[recpath_id].sequence[m+1]=r[i].ray[j].source.path[path_id].sequence[2*r[i].ray[j].source.path[path_id].n_tf-1]
            l=(m+1)/2
            s[r[i].source_equivalent].path[recpath_id].vtype_sequence[m]=r[i].ray[j].source.path[path_id].vtype_sequence[2*r[i].ray[j].source.path[path_id].n_tf-l+1]

            if (m>1):
                r[i].ray[j].source.path[path_id].sequence[2*r[i].ray[j].source.path[path_id].n_tf-m+1]
                if (s[r[i].source_equivalent].path[recpath_id].sequence[m+1]==s[r[i].source_equivalent].path[recpath_id].sequence[m-1]):
                    j=s[r[i].source_equivalent].path[recpath_id].sequence[m+1]
                    s[r[i].source_equivalent].path[recpath_id].sequence[m+1]=s[r[i].source_equivalent].path(recpath_id).sequence[m]
                    s[r[i].source_equivalent].path[recpath_id].sequence[m]=j
        s[r[i].source_equivalent].path[recpath_id].sequence[1]=0
        ipp_receivers[i]+=1

# Line (671)

for i in range(0,n_sources):
    for j in range(0,s[i].n_paths):
        if (s[i].path[j].refstep!=0):
            s[i].path[j].n_tf=s[i].path[j].refstep - 1

for i in range(0,n_sources_ppinc):
    x=n_vtypes*n_regions
    for j in range(0,s[i].n_paths):
        x+=s[i].path[j].n_tf

    s[i].time_field=np.zeros(x)
    for k in range(0,x):
        time_field_defaults(s[i].time_field[k])
        s[i].time_field[j].id=k

for i in range(0,n_interfaces):
    display_interface(intersection[i])

display_sources()
display_receivers()

initialise_inversion(do_frechet_derivatives)  #do_frechet_derivatives is logical in fortran

for i in range(0,n_sources_ppinc):
    if (s[i].is_local):
        initialise_source_regions(s[i])
    if (s[i].is_teleseismic):
        initialise_teleseismic_source(s[i])
    for j in range(0,s[i].n_paths):
        vtype=s[i].path[j].vtype_sequence[1]
        if (s[i].is_local):
            if (s[i].on_interface):
                if (s[i].path[j].sequence[2]==s[i].topint_id - 1):
                    prev_tf=s[i].first_tf_up[vtype]
                if (s[i].path[j].sequence[2]==s[i].topint_id + 1):
                    prev_tf=s[i].first_tf_down[vtype]
        if (s[i].is_teleseismic):
            prev_tf=s[i].first_tf_up[vtype]
        s[i].path[j].tf_sequence[1]=prev_tf
        #Line 873
        for m in np.arange(3,2*s[i].path[j].n_tf,2):
            step=(m+1)/2
            vtype=s[i].path[j].vtype_sequence[step]

            # Pointer definition:
            istar=intersection[s[i].path[j].sequence[m]]
            inext=intersection[s[i].path[j].sequence[m+1]]

            if (istart.id>inext.id):
                reg=istart.regabo
            else:
                reg=istart.regbel

            print((a5,i4,a5,i4,a16,i4,a5,i4,a16,i4), 'path',s[i].path[j].id,'leg',step,'from interface',istart.id,'to',inext.id,'through region',reg.id)
            if (step>2):
                if (istart.id==s[i].time_field[prev_tf].inonstart.id):
                    if (istart.id>inext.id):
                        ctype=1+4*(vtype-1)
                    if (istart.id<inext.id):
                        ctype=2+4*(vtype-1)
                else:
                    if(~s[i].time_field[prev_tf].turning_rays_present):
                        print((a5,i4,a5,i4,a4), 'path',j,'leg',step,'no turning rays present, non-existing path')
                        s[i].path[j].valid=False
                        print('Step=',step,s[i].path[j].sequence[1],prev_tf,s[i].time_field[prev_tf].inonstart.id)

                    if (istart.id>inext.id):
                        ctype=3+4*(vtype-1)
                    if (istart.id<inext.id):
                        ctype=4+4*(vtype-1)
            else:
                print('Previous source is a source time-field')
                if (istart.id==s[i].time_field[prev_tf].reg.itop.id):
                    if (istart.id>inext.id):
                        ctype=1+(vtype-1)*4
                    if (istart.id<inext.id):
                        ctype=2+(vtype-1)*4
                else:
                    if (istart.id>inext.id):
                        ctype=3+4*(vtype-1)
                    if (istart.id<inext.id):
                        ctype=4+4*(vtype-1)
            if (s[i].time_field[prev_tf].next_tf[ctype]==0):
                if (istart.id==s[i].time_field[prev_tf].reg.ibot.id):
                    k=s[i].time_field[prev_tf].reg.nnode-istart.nnode+1
                    l=s[i].time_field[prev_tf].reg.nnode
                    istart.arrivaltime=s[i].time_field[prev_tf].arrivaltime[i:j]
                    istart.time_gradient[1][:]=s[i].time_field[prev_tf].time_gradient[1][i:j]
                    istart.time_gradient[2][:]=s[i].time_field[prev_tf].time_gradient[2][i:j]
                    istart.time_gradient[3][:]=s[i].time_field[prev_tf].time_gradient[3][i:j]
                else:
                    k=s[i].time_field[prev_tf].reg.ngnode+1
                    l=s[i].time_field[prev_tf].reg.ngnode+istart.nnode
                    istart.arrivaltime=s[i].time_field[prev_tf].arrivaltime[i:j]
                    istart.time_gradient[1][:]=s[i].time_field[prev_tf].time_gradient[1][i:j]
                    istart.time_gradient[2][:]=s[i].time_field[prev_tf].time_gradient[2][i:j]
                    istart.time_gradient[3][:]=s[i].time_field[prev_tf].time_gradient[3][i:j]
                if (reg.id==s[i].time_field[prev_tf].reg.id):
                    reflect_gradient(istart.s[i].time_field[prev_tf],vtype)
                else:
                    direction=s[i].time_field[prev_tf].reg.id-reg_id
                    reflect_gradient(istart.s[i].time_field[prev_tf],vtype,direction)
                sweep_region_from_interface(reg,istart,vtype,s[i])
                s[i].time_field[s[i].n_time_fields].prev_tf=prev_tf
                s[i].time_field[prev_tf].next_tf[ctype]=s[i].n_time_fields
                s[i].path[j].tf_sequence[step]=s[i].n_time_fields

                s[i].time_field[s[i].n_time_fields].istart=istart
                s[i].time_field[s[i].n_time_fields].inonstart=inext
                s[i].time_field[s[i].n_time_fields].reg=reg
                s[i].time_field[s[i].n_time_fields].vtype=vtype

                prev_tf=s[i].n_time_fields
                print('Created new timefields:',s[i].n_time_fields)
            else:
                prev_tf=s[i].time_field[prev_tf].next_tf[ctype]
                print('Used existing time field',prev_tf)
                s[i].path[j].tf_sequence[step]=prev_tf

        print('(a5,i5,a12,10i5)','path',s[i].path[j].id,'timefields',s[i].path[j].tf_sequence[1:s[i].path[j]].n_tf)
    if (save_timefields_mode):
        for j in range(0,s[i].n_paths):
            if(s[i].path[j].gridsave):
                write_arrivaltime_grid(s[i],s[i].path[j])

    if (file_mode):
        global_source_counter+=1
        nfile=global_source_counter+1000
        s[i].nfile=nfile
        open(nfile,form='unformatted')
#         for j in range(0,s[i].n_time_fields):
#             write(nfile) s[i].time_field[j].arrivaltime
#             write(nfile) s[i].time_field[j].time_gradient
    # write data from files

    if (no_pp_mode):
        print("Starting the ray tracing for source: ",s[i].id)
        for j in range(0,r[i].n_rays):
            if (r[i].ray[j].source.id!=s[i].id):
                path_id=r[i].ray[j].raypath_id
            if (s[i].path[path_id].valid):
                if (s[i].path[path_id].refstep==0):
                    trace_ray_from_receiver(r[i],s[i],r[i].ray[j])
                    if (r[i].ray[j].valid):
                        print('(a12,i4,a10,i4,a15,i4,a4,f10.4,2l5) traced ray',j,'to source',s[i].id,'from receiver',i,'t=',r[i].ray[j].receiver_time,r[i].ray[j].diffracted,r[i].ray[j].headwave)
                        y=0
                        t_arrival=-1.0
                        #line 1123
                if (s[i].path[path_id].refstep!=0):
                    trace_reflectionfit(n,m)

            if (do_frechet_derivatives):
                if (r[i].ray[j].is_multiray):
                    print("Inconsistency")
                else:
                    if (r[i].ray[j].valid):
                        print("getting partials for rec",i,"ray",j)
                        ray_partials(r[i].ray[j])
                    else:
                        print("No valid ray path for rec",i,"ray",j)
                    write_frechet_derivatives(i,j)
            if (save_rays_mode):
                write_valid_rays(i,j)

if (display_mode):
    display_stored_rays

if (np.logical_and(n_receivers>0,(~no_pp_mode))):
    initialise_inversion(do_frechet_derivatives)
    raypoint_counter=0
    print("Starting the ray tracing")
for i in range(0,n_receivers):
    for j in range(0,r[i].n_rays):
        path_id=r[i].ray[j].raypath_id
        if (r[i].ray[j].source.path[path_id].valid):
            if(r[i].ray[j].source.path[path_id].refstep==0):
                if(filemode):
                    load_source_timefields(r[i].ray[j].source)
                trace_ray_from_receiver(r[i],r[i].ray[j].source,r[i].ray[j])
                if(r[i].ray[j].valid):
                    print('(a12,i4,a10,i4,a15,i4,a4,f10.4,2l5) traced ray',j,'to source',r[i].ray[j].source_id,'from receiver',i,'t=',r[i].ray[j].receiver_time,r[i].ray[j].diffracted,r[i].ray[j].headwave)
                    k=0
#                     write(11,'(4i6,f15.6,2l5)') i,r[i].ray[j].source_id,j,k,r[i].ray[j].receiver_time,r[i].ray[j].diffracted,r[i].ray[j].headwave

                    if(display_mode):
                        store_ray(r[i].ray[j])
                else:
                    print('(a12,i4,a10,i4,a15,i4,a13)ray,',j,'to source',r[i].ray[j].source_id,' from receiver',i,' is invalid')
                    k=0
                    t_arrival=-1.0
#                     write(11,'(4i6,f15.6,2l5)') i,r[i].ray[j].source_id,j,k,r[i].ray[j].receiver_time,r[i].ray[j].diffracted,r[i].ray[j].headwave
            if(do_frechet_derivatives):
                if(r[i].ray[j].is_multiray):
                    for k in range(0,r[i].ray[j].n_subrays):
                        if(r[i].ray[j].subray[k]):
                            print("Getting partials for rec",i,"ray",j,"subray",k)
                            ray_partials(r[i].ray[j].subray[k])
                        else:
                            print("No valid ray path for rec",i,"ray",j,"subray",k)
                else:
                    if(r[i].ray[j].valid):
                        print("Getting partials for rec",i,"ray",j)
                        ray_partials(r[i].ray[j])
                    else:
                        print("No valid ray path for rec",i,"ray",j)
                write_frechet_derivatives(i,j)
        else:
            print("ray",j,"to receiver",i,": requested path was recognised as invalid during the timefield calculations")
            r[i].ray[j].valid = False
            k=0
            t_arrival=-1.0
#             write(11,'(4i6,f15.6,2l5)') i,r[i].ray[j].source_id,j,k,t_arrival,r[i].ray[j].diffracted,r[i].ray[j].headwave
        if(save_rays_mode):
            write_valid_rays(i,j)

if(display_mode):
    display_stored_rays()

IndexError: index 5 is out of bounds for axis 0 with size 5